<a href="https://colab.research.google.com/github/itihub/jupyter_learn/blob/main/supply_chain_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 供应链网络优化模型

本 Notebook 旨在构建一个快消品供应链网络优化模型，目标是最小化总成本（生产、运输、库存和惩罚成本）。模型考虑了工厂、分仓、客户之间的多层级关系，以及多周期生产和带有最小/最大产值约束的生产线策略。

我们将使用 `pulp` 库进行线性规划，并使用 `pandas`, `plotly` 和 `folium` 进行数据分析和可视化。


强大的决策支持工具，可以从中挖掘出远超“最优解”本身的商业价值数据和风控数据。以下是你可以从模型结果中提取和分析的一些关键洞察，这些都对商业决策和风险管理至关重要：
1. 商业价值数据
这些数据可以帮助你优化运营策略，提高利润率。

+ 资源利用率：
    + 生产线利用率：通过计算 value(produce) 除以生产线的最大产能，你可以知道每条生产线在每个周期的繁忙程度。这有助于你决定是否需要升级、停用或投资新生产线。
    + 仓储空间利用率：通过分析 value(factory_inventory) 和 value(warehouse_inventory)，你可以计算出库存的平均占用率。这有助于评估现有仓储能力是否足够，或者是否需要调整租金、寻找新仓库。

+ 成本效率分析：
    + 单位成本：模型可以计算出每件产品从生产到最终交付给客户的总单位成本。这可以作为定价策略的依据，并帮助你识别成本最高的配送路径或生产线。
    + 成本分解细化：你已经实现了总成本的分解。但你可以更进一步，按产品、按客户或按周期来分解成本，找出哪些产品或客户是利润的主要来源，或者哪些时期成本最高。

+ 库存策略优化：
    + 安全库存量：通过分析模型中的库存水平，你可以了解在满足所有需求的前提下，模型如何管理库存。这可以帮助你设定合理的安全库存量，以平衡库存成本和缺货风险。
    + 提前生产策略：如果模型在需求较低的周期安排了大量生产，并将产品存入仓库，这说明模型认为提前生产比在需求高峰期紧急生产更经济。你可以将此作为季节性或周期性生产的指导方针。

2. 风控数据
这些数据可以帮助你评估和管理供应链中的潜在风险，为制定应急预案提供依据。

+ 单点依赖风险：
    + 工厂生产集中度：分析 produce 变量，如果某个工厂承担了绝大多数产品的生产，那么该工厂一旦停工，将对整个供应链造成巨大冲击。模型可以量化这种集中度。
    + 配送路径集中度：分析 flow 变量，如果某个分仓或运输路线承担了绝大部分的配送任务，那么这条路径的任何中断（如交通管制、自然灾害）都可能导致大规模延误。

+ 需求满足风险：
    + 缺货惩罚成本：你的模型包含了 penalty_cost_val。这个值如果非零，就意味着模型在最优解中允许了部分缺货，这本身就是一种风险信号。你可以通过分析是哪个客户、哪个周期、哪种产品出现了缺货，来识别最脆弱的环节。

+ 产能瓶颈识别：
    + 最大生产量约束：如果某个工厂的生产量 value(produce) 总是接近其最大产能约束，这表明该工厂是整个供应链的瓶颈。你可以将此作为投资扩建或寻找备选供应商的决策依据。

In [14]:
!pip install pulp
!pip install pandas
!pip install plotly
!pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 68.5 MB/s eta 0:00:00


In [15]:
# 导入所需库
from pulp import *
import pandas as pd
from collections import defaultdict
import plotly.express as px
import folium
from folium.plugins import MarkerCluster, Fullscreen, PolyLineTextPath
from IPython.display import display


## 数据准备

In [16]:
# --- 1. 数据准备 ---

# 定义所有站点、产品和周期
factories = ['北京工厂', '广州工厂', '上海工厂'] # 工厂列表
warehouses = ['上海分仓', '武汉分仓', '成都分仓'] # 分仓列表
customers = ['沈阳客户', '深圳客户', '成都客户', '西安客户', '杭州客户'] # 客户列表
periods = [1, 2, 3]  # 三旬，每个周期代表一旬 (约10天)
days_per_period = 10 # 每旬10天
products = ['橙汁', '苹果汁', '葡萄汁', '芒果汁'] # 产品列表

# 产品数据（售价、重量、体积
product_data = {
    # '产品名称': {'sales_price': 销售价格 (元/件), 'weight': 重量 (kg/件), 'volume': 体积 (m^3/件)}
    '橙汁': {'sales_price': 8.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '苹果汁': {'sales_price': 7.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '葡萄汁': {'sales_price': 9.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '芒果汁': {'sales_price': 10.0, 'weight': 0.5, 'volume': 0.0005} # 假设每件500ml
}

# 生产线信息
production_lines_info = {
    '北京工厂': {
        'Line_1': {
            'products': ['橙汁', '葡萄汁'], # 该生产线可以生产的产品
            'min_daily_output': {'橙汁': 2000, '葡萄汁': 1800}, # 每日最小产值 (件/天)
            'max_daily_output': {'橙汁': 5000, '葡萄汁': 4500} # 每日最大产值 (件/天)
        },
        'Line_2': {
            'products': ['苹果汁', '芒果汁'],
            'min_daily_output': {'苹果汁': 1500, '芒果汁': 1200},
            'max_daily_output': {'苹果汁': 4000, '芒果汁': 3000}
        }
    },
    '广州工厂': {
        'Line_A': {
            'products': ['橙汁', '芒果汁'],
            'min_daily_output': {'橙汁': 1500, '芒果汁': 1000},
            'max_daily_output': {'橙汁': 6000, '芒果汁': 5000}
        },
        'Line_B': {
            'products': ['苹果汁', '葡萄汁'],
            'min_daily_output': {'苹果汁': 1800, '葡萄汁': 1600},
            'max_daily_output': {'苹果汁': 6500, '葡萄汁': 5500}
        }
    },
    '上海工厂': {
        'Line_X': {
            'products': ['橙汁', '苹果汁', '葡萄汁', '芒果汁'],
            'min_daily_output': {'橙汁': 1000, '苹果汁': 800, '葡萄汁': 900, '芒果汁': 700},
            'max_daily_output': {'橙汁': 4000, '苹果汁': 3500, '葡萄汁': 3800, '芒果汁': 3000}
        }
    }
}

all_lines = list(set([line for f in factories for line in production_lines_info[f]])) # 所有生产线的集合


# 成本数据 (单位: 元/件)
# 生产成本: 每件产品在特定工厂的特定生产线上生产的成本 (元/件)
# 成本包含原料、人工及其他变动生产成本。
production_cost = {
    ('北京工厂', 'Line_1', '橙汁'): 1.5 + 1.0 + 2.0, # 原料 + 人工 + 其他变动成本
    ('北京工厂', 'Line_1', '葡萄汁'): 1.8 + 1.1 + 2.2,
    ('北京工厂', 'Line_2', '苹果汁'): 1.2 + 0.9 + 1.8,
    ('北京工厂', 'Line_2', '芒果汁'): 2.0 + 1.2 + 2.5,
    ('广州工厂', 'Line_A', '橙汁'): 1.5 + 1.0 + 1.9,
    ('广州工厂', 'Line_A', '芒果汁'): 2.0 + 1.2 + 2.4,
    ('广州工厂', 'Line_B', '苹果汁'): 1.2 + 0.9 + 1.7,
    ('广州工厂', 'Line_B', '葡萄汁'): 1.8 + 1.1 + 2.1,
    ('上海工厂', 'Line_X', '橙汁'): 1.5 + 1.0 + 2.1,
    ('上海工厂', 'Line_X', '苹果汁'): 1.2 + 0.9 + 1.9,
    ('上海工厂', 'Line_X', '葡萄汁'): 1.8 + 1.1 + 2.3,
    ('上海工厂', 'Line_X', '芒果汁'): 2.0 + 1.2 + 2.6
}

# 生产线激活成本 (单位: 元/旬)
# 每旬激活一条生产线所需的固定成本 (元/旬)
line_activation_cost = {
    ('北京工厂', 'Line_1'): 5000, ('北京工厂', 'Line_2'): 4800,
    ('广州工厂', 'Line_A'): 4500, ('广州工厂', 'Line_B'): 4000,
    ('上海工厂', 'Line_X'): 6000
}


# 仓储成本 (单位: 元/件/旬)
# 每件产品在工厂仓库中存放一旬的成本 (元/件/旬)
factory_inventory_cost = {
    '北京工厂': {'橙汁': 0.08, '苹果汁': 0.10, '葡萄汁': 0.09, '芒果汁': 0.11},
    '广州工厂': {'橙汁': 0.07, '苹果汁': 0.09, '葡萄汁': 0.08, '芒果汁': 0.10},
    '上海工厂': {'橙汁': 0.09, '苹果汁': 0.11, '葡萄汁': 0.10, '芒果汁': 0.12}
}

# 每件产品在分仓中存放一旬的成本 (元/件/旬)
warehouse_inventory_cost = {
    '上海分仓': {'橙汁': 0.1, '苹果汁': 0.12, '葡萄汁': 0.11, '芒果汁': 0.13},
    '武汉分仓': {'橙汁': 0.09, '苹果汁': 0.11, '葡萄汁': 0.10, '芒果汁': 0.12},
    '成都分仓': {'橙汁': 0.11, '苹果汁': 0.13, '葡萄汁': 0.12, '芒果汁': 0.14}
}

# 运输成本 (单位: 元/件)
# 工厂到分仓的运输成本 (元/件)
transport_cost_fw = {
    ('北京工厂', '上海分仓'): 0.5, ('北京工厂', '武汉分仓'): 0.6, ('北京工厂', '成都分仓'): 0.9,
    ('广州工厂', '上海分仓'): 0.4, ('广州工厂', '武汉分仓'): 0.3, ('广州工厂', '成都分仓'): 0.7,
    ('上海工厂', '上海分仓'): 0.1, ('上海工厂', '武汉分仓'): 0.5, ('上海工厂', '成都分仓'): 0.8
}
# 工厂到客户的运输成本 (元/件)
transport_cost_fc = {
    ('北京工厂', '沈阳客户'): 0.2, ('广州工厂', '沈阳客户'): 1.5, ('上海工厂', '沈阳客户'): 0.8,
    ('北京工厂', '深圳客户'): 1.8, ('广州工厂', '深圳客户'): 0.2, ('上海工厂', '深圳客户'): 0.7,
    ('北京工厂', '成都客户'): 1.2, ('广州工厂', '成都客户'): 0.9, ('上海工厂', '成都客户'): 0.4,
    ('北京工厂', '西安客户'): 0.7, ('广州工厂', '西安客户'): 1.1, ('上海工厂', '西安客户'): 0.6,
    ('北京工厂', '杭州客户'): 0.9, ('广州工厂', '杭州客户'): 0.8, ('上海工厂', '杭州客户'): 0.3
}
# 分仓到客户的运输成本 (元/件)
transport_cost_wc = {
    ('上海分仓', '沈阳客户'): 0.8, ('武汉分仓', '沈阳客户'): 1.3, ('成都分仓', '沈阳客户'): 1.5,
    ('上海分仓', '深圳客户'): 0.6, ('武汉分仓', '深圳客户'): 0.7, ('成都分仓', '成都客户'): 1.0,
    ('上海分仓', '成都客户'): 0.5, ('武汉分仓', '成都客户'): 0.4, ('成都分仓', '成都客户'): 0.2,
    ('上海分仓', '西安客户'): 0.3, ('武汉分仓', '西安客户'): 0.5, ('成都分仓', '西安客户'): 0.3,
    ('上海分仓', '杭州客户'): 0.2, ('武汉分仓', '杭州客户'): 0.6, ('成都分仓', '杭州客户'): 0.9
}


# 客户需求数据 (单位: 件)
# 每个客户在每个周期对每种产品的需求量 (件)
demand = {
    '沈阳客户': {'橙汁': {1: 10000, 2: 12000, 3: 8000}, '苹果汁': {1: 5000, 2: 6000, 3: 7000},
               '葡萄汁': {1: 3000, 2: 4000, 3: 3500}, '芒果汁': {1: 2000, 2: 2500, 3: 2200}},
    '深圳客户': {'橙汁': {1: 15000, 2: 18000, 3: 16000}, '苹果汁': {1: 8000, 2: 9000, 3: 10000},
               '葡萄汁': {1: 6000, 2: 7000, 3: 6500}, '芒果汁': {1: 5000, 2: 5500, 3: 5200}},
    '成都客户': {'橙汁': {1: 12000, 2: 10000, 3: 14000}, '苹果汁': {1: 7000, 2: 6000, 3: 8000},
               '葡萄汁': {1: 5000, 2: 4500, 3: 5500}, '芒果汁': {1: 4000, 2: 3500, 3: 4200}},
    '西安客户': {'橙汁': {1: 8000, 2: 9000, 3: 7000}, '苹果汁': {1: 4000, 2: 4500, 3: 3800},
               '葡萄汁': {1: 3000, 2: 3200, 3: 2800}, '芒果汁': {1: 2500, 2: 2800, 3: 2400}},
    '杭州客户': {'橙汁': {1: 9000, 2: 8500, 3: 9500}, '苹果汁': {1: 5000, 2: 4800, 3: 5200},
               '葡萄汁': {1: 4000, 2: 3800, 3: 4200}, '芒果汁': {1: 3000, 2: 2800, 3: 3200}}
}


# 惩罚成本 (单位: 元/件)
# 每件未满足需求的惩罚成本 (元/件)
penalty_cost_per_unit = 100

# 地理位置数据 (用于地图可视化)
# 各个地点对应的经纬度 (纬度, 经度)
locations = {
    '北京工厂': (39.9042, 116.4074), '广州工厂': (23.1291, 113.2644), '上海工厂': (31.2304, 121.4737),
    '上海分仓': (31.2304, 121.4737), '武汉分仓': (30.5928, 114.3055), '成都分仓': (30.5728, 104.0668),
    '沈阳客户': (41.7963, 123.4315), '深圳客户': (22.5431, 114.0579), '成都客户': (30.5728, 104.0668),
    '西安客户': (34.2632, 108.9525), '杭州客户': (30.2500, 120.1600)
}

## 数据建模

In [19]:
# --- 2. 模型构建与求解 ---

"""构建并求解供应链优化模型"""
model = LpProblem("Supply_Chain_Optimization", LpMinimize)

# 决策变量
line_on = LpVariable.dicts("Line_On", (factories, all_lines, products, periods), 0, 1, LpBinary)
produce = LpVariable.dicts("Produce", (factories, all_lines, products, periods), 0, None, LpContinuous)
flow_fw = LpVariable.dicts("Flow_FW", (factories, warehouses, products, periods), 0, None, LpContinuous)
flow_fc = LpVariable.dicts("Flow_FC", (factories, customers, products, periods), 0, None, LpContinuous)
flow_wc = LpVariable.dicts("Flow_WC", (warehouses, customers, products, periods), 0, None, LpContinuous)
factory_inventory = LpVariable.dicts("Factory_Inventory", (factories, products, periods), 0, None, LpContinuous)
warehouse_inventory = LpVariable.dicts("Warehouse_Inventory", (warehouses, products, periods), 0, None, LpContinuous)

# 目标函数：最小化总成本
total_cost = []
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if p in production_lines_info[f][line]['products']:
                    total_cost.append(production_cost[(f, line, p)] * produce[f][line][p][t])
                    total_cost.append(line_activation_cost[(f, line)] * line_on[f][line][p][t])
for f in factories:
    for p in products:
        for t in periods:
            total_cost.append(factory_inventory_cost[f][p] * factory_inventory[f][p][t])
for w in warehouses:
    for p in products:
        for t in periods:
            total_cost.append(warehouse_inventory_cost[w][p] * warehouse_inventory[w][p][t])
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_fw[(f, w)] * flow_fw[f][w][p][t])
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_fc.get((f, c), 0) * flow_fc[f][c][p][t])
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_wc.get((w, c), 0) * flow_wc[w][c][p][t])
for c in customers:
    for p in products:
        for t in periods:
            unmet_demand_var = LpVariable(f"Unmet_Demand_{c}_{p}_{t}", 0, None)
            received_flow = lpSum([flow_fc[f][c][p][t] for f in factories]) + lpSum([flow_wc[w][c][p][t] for w in warehouses])
            model += unmet_demand_var >= demand[c][p][t] - received_flow
            total_cost.append(penalty_cost_per_unit * unmet_demand_var)

model += lpSum(total_cost)

# 约束条件
for c in customers:
    for p in products:
        for t in periods:
            model += lpSum([flow_fc[f][c][p][t] for f in factories] + [flow_wc[w][c][p][t] for w in warehouses]) >= demand[c][p][t]

for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if p in production_lines_info[f][line]['products']:
                    min_val = production_lines_info[f][line]['min_daily_output'][p] * days_per_period
                    max_val = production_lines_info[f][line]['max_daily_output'][p] * days_per_period
                    model += produce[f][line][p][t] >= min_val * line_on[f][line][p][t]
                    model += produce[f][line][p][t] <= max_val * line_on[f][line][p][t]
                else:
                    model += produce[f][line][p][t] == 0
                    model += line_on[f][line][p][t] == 0

for f in factories:
    for line in production_lines_info[f]:
        for t in periods:
            model += lpSum([line_on[f][line][p][t] for p in products]) <= 1

for f in factories:
    for p in products:
        for t in periods:
            total_produced = lpSum([produce[f][line][p][t] for line in production_lines_info[f]])
            total_shipped = lpSum([flow_fw[f][w][p][t] for w in warehouses] + [flow_fc[f][c][p][t] for c in customers])
            if t == periods[0]:
                model += factory_inventory[f][p][t] == total_produced - total_shipped
            else:
                model += factory_inventory[f][p][t] == factory_inventory[f][p][t-1] + total_produced - total_shipped

for w in warehouses:
    for p in products:
        for t in periods:
            inflow = lpSum([flow_fw[f][w][p][t] for f in factories])
            outflow = lpSum([flow_wc[w][c][p][t] for c in customers])
            if t == periods[0]:
                model += warehouse_inventory[w][p][t] == inflow - outflow
            else:
                model += warehouse_inventory[w][p][t] == warehouse_inventory[w][p][t-1] + inflow - outflow

# 求解模型
model.solve()
if LpStatus[model.status] != 'Optimal':
    print("模型求解失败，未找到最优解。")

## 成本可视化

In [20]:
# --- 3. 结果分析与可视化 ---

"""分析模型结果并进行可视化"""
print(f"总成本：{value(model.objective):,.2f} 元")

# 提取成本数据
# 生产成本
prod_cost_val = 0
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if value(produce[f][line][p][t]) > 0:
                    prod_cost_val += value(produce[f][line][p][t]) * production_cost[(f, line, p)]

# 生产线激活成本
act_cost_val = 0
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if value(line_on[f][line][p][t]) > 0:
                    act_cost_val += value(line_on[f][line][p][t]) * line_activation_cost[(f, line)]

# 运输成本
transport_cost_val = 0
# 工厂到分仓
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                if value(flow_fw[f][w][p][t]) > 0:
                    transport_cost_val += value(flow_fw[f][w][p][t]) * transport_cost_fw.get((f, w), 0)
# 工厂到客户
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_fc[f][c][p][t]) > 0:
                    transport_cost_val += value(flow_fc[f][c][p][t]) * transport_cost_fc.get((f, c), 0)
# 分仓到客户
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_wc[w][c][p][t]) > 0:
                    transport_cost_val += value(flow_wc[w][c][p][t]) * transport_cost_wc.get((w, c), 0)

# 工厂库存成本
factory_inv_cost_val = 0
for f in factories:
    for p in products:
        for t in periods:
            if value(factory_inventory[f][p][t]) > 0:
                factory_inv_cost_val += value(factory_inventory[f][p][t]) * factory_inventory_cost[f][p]

# 分仓库存成本
warehouse_inv_cost_val = 0
for w in warehouses:
    for p in products:
        for t in periods:
            if value(warehouse_inventory[w][p][t]) > 0:
                warehouse_inv_cost_val += value(warehouse_inventory[w][p][t]) * warehouse_inventory_cost[w][p]

# 惩罚成本
penalty_cost_val = 0
for c in customers:
    for p in products:
        for t in periods:
            unmet_demand_var = model.variablesDict()[f"Unmet_Demand_{c}_{p}_{t}"]
            if value(unmet_demand_var) > 0:
                penalty_cost_val += value(unmet_demand_var) * penalty_cost_per_unit
    cost_breakdown = {
        '生产成本': prod_cost_val,
        '运输成本': transport_cost_val,
        '库存成本': factory_inv_cost_val + warehouse_inv_cost_val,
        '生产线激活成本': act_cost_val,
        '惩罚成本': penalty_cost_val
    }

# 可视化1: 成本分解饼图
cost_df = pd.DataFrame(list(cost_breakdown.items()), columns=['Cost Type', 'Amount'])
fig1 = px.pie(cost_df, values='Amount', names='Cost Type', title='总成本分解')
fig1.show()


总成本：1,956,404.00 元


## 供应链的生产-需求平衡

In [21]:
# 1. 提取工厂每旬每种产品的产量
production_data_fine = []
for f in factories:
    for p in products:
        for t in periods:
            total_produced = sum(
                value(produce[f][line][p][t])
                for line in production_lines_info[f]
            )
            if total_produced > 0:
                production_data_fine.append({
                    '地点': f,
                    '周期（旬）': t,
                    '产品': p,
                    '数量': total_produced,
                    '类型': '产量'
                })

# 2. 提取客户每旬每种产品的需求量
demand_data_fine = []
for c in customers:
    for p in products:
        for t in periods:
            total_demanded = demand[c][p][t]
            if total_demanded > 0:
                demand_data_fine.append({
                    '地点': c,
                    '周期（旬）': t,
                    '产品': p,
                    '数量': total_demanded,
                    '类型': '需求量'
                })

# 合并生产和需求细化数据
production_demand_fine_df = pd.DataFrame(production_data_fine + demand_data_fine)

print("\n--- 工厂/客户产量与需求对照（细化到产品） ---")
if not production_demand_fine_df.empty:
    display(production_demand_fine_df)

    # --- 可视化：细化到产品的产量与需求对照 ---
    # 使用 facet_col 来按地点切分，使用 color 来区分产品
    fig = px.bar(
        production_demand_fine_df,
        x='周期（旬）',
        y='数量',
        color='产品',
        facet_col='地点',
        barmode='group',
        title='细化到产品的产量 vs 需求量',
        height=400,
        labels={'数量': '数量 (件)', '地点': '地点', '产品': '产品'}
    )
    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig.show()

    # 另一个更清晰的方案：按产品切分
    fig2 = px.bar(
        production_demand_fine_df,
        x='周期（旬）',
        y='数量',
        color='类型',
        facet_row='产品', # 按产品切分
        facet_col='地点', # 按地点切分
        barmode='group',
        title='产量 vs 需求量（按产品和地点）',
        height=600,
        labels={'数量': '数量 (件)', '地点': '地点', '类型': '类型'}
    )
    fig2.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig2.show()

else:
    print("没有可展示的细化产量与需求数据。")


--- 工厂/客户产量与需求对照（细化到产品） ---


,地点,周期（旬）,产品,数量,类型
0,北京工厂,1,橙汁,32000.0,产量
1,北京工厂,2,苹果汁,15000.0,产量
2,北京工厂,2,葡萄汁,18000.0,产量
3,北京工厂,1,芒果汁,21200.0,产量
4,广州工厂,1,橙汁,60000.0,产量
...,...,...,...,...,...
66,杭州客户,2,葡萄汁,3800.0,需求量
67,杭州客户,3,葡萄汁,4200.0,需求量
68,杭州客户,1,芒果汁,3000.0,需求量
69,杭州客户,2,芒果汁,2800.0,需求量


## 最优生产计划

In [28]:
print("\n--- 最优生产计划 ---")
# 提取生产计划数据
production_plan_data = []
# 遍历所有生产相关的变量
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                # 只有当生产量大于0时才打印，忽略未生产的情况
                if value(produce[f][line][p][t]) > 0:
                    plan_entry = {
                        '工厂': f,
                        '生产线': line,
                        '产品': p,
                        '周期（旬）': t,
                        '生产量': int(value(produce[f][line][p][t])),
                        '单位生产成本': production_cost.get((f, line, p), 0)
                    }
                    production_plan_data.append(plan_entry)

production_plan_df = pd.DataFrame(production_plan_data)

if not production_plan_df.empty:
    # print(production_plan_df.to_string(index=False))
    display(production_plan_df)
else:
    print("没有生产记录。")

# 可视化3: 最优生产计划条形图
if not production_plan_df.empty:
    fig3 = px.bar(
        production_plan_df,
        x='工厂',
        y='生产量',
        color='产品',
        barmode='group',
        title='最优生产计划（各工厂生产量）',
        labels={'生产量': '生产量', '工厂': '工厂'}
    )
    fig3.show()


--- 最优生产计划 ---


,工厂,生产线,产品,周期（旬）,生产量,单位生产成本
0,北京工厂,Line_1,橙汁,1,32000,4.5
1,北京工厂,Line_1,葡萄汁,2,18000,5.1
2,北京工厂,Line_2,苹果汁,2,15000,3.9
3,北京工厂,Line_2,芒果汁,1,21200,5.7
4,广州工厂,Line_A,橙汁,1,60000,4.4
5,广州工厂,Line_A,橙汁,3,34000,4.4
6,广州工厂,Line_A,芒果汁,2,29600,5.6
7,广州工厂,Line_B,苹果汁,2,49300,3.8
8,广州工厂,Line_B,葡萄汁,1,48000,5.0
9,上海工厂,Line_X,橙汁,2,40000,4.6


## 最优配送方案

In [29]:
# --- 提取配送方案数据 ---
flow_data = []
# 工厂到分仓
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                if value(flow_fw[f][w][p][t]) > 0:
                    flow_data.append({
                        '起点': f,
                        '终点': w,
                        '产品': p,
                        '周期（旬）': t,
                        '数量': int(value(flow_fw[f][w][p][t])),
                        '类型': '工厂->分仓'
                    })
# 工厂到客户
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_fc[f][c][p][t]) > 0:
                    flow_data.append({
                        '起点': f,
                        '终点': c,
                        '产品': p,
                        '周期（旬）': t,
                        '数量': int(value(flow_fc[f][c][p][t])),
                        '类型': '工厂->客户'
                    })
# 分仓到客户
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_wc[w][c][p][t]) > 0:
                    flow_data.append({
                        '起点': w,
                        '终点': c,
                        '产品': p,
                        '周期（旬）': t,
                        '数量': int(value(flow_wc[w][c][p][t])),
                        '类型': '分仓->客户'
                    })

flow_df = pd.DataFrame(flow_data)

# --- 1. 最优配送方案数据表格 ---
print("\n--- 最优配送方案 ---")
if not flow_df.empty:
    # print(flow_df.to_string(index=False))
    display(flow_df)
else:
    print("没有配送记录。")

# --- 2. 配送流向地图 ---
if not flow_df.empty:
    m = folium.Map(location=[34.7, 108.9], zoom_start=4)
    # 定义图标配置
    icon_configs = {
        'factories': {
            'icon': 'industry',
            'prefix': 'fa',
            'color': 'red',
            'icon_color': 'white',
            'tooltip': '工厂'
        },
        'warehouses': {
            'icon': 'warehouse',
            'prefix': 'fa',
            'color': 'blue',
            'icon_color': 'white',
            'tooltip': '分仓'
        },
        'customers': {
            'icon': 'user',
            'prefix': 'fa',
            'color': 'green',
            'icon_color': 'white',
            'tooltip': '客户'
        }
    }

    # 添加站点标记，并使用不同的图标
    marker_cluster = MarkerCluster().add_to(m)
    for name, coords in locations.items():
        # 根据站点类型选择图标配置
        if name in factories:
            config = icon_configs['factories']
        elif name in warehouses:
            config = icon_configs['warehouses']
        else: # 默认为客户
            config = icon_configs['customers']

        # 创建一个自定义图标
        custom_icon = folium.Icon(
            icon=config['icon'],
            prefix=config['prefix'],
            color=config['color'],
            icon_color=config['icon_color']
        )

        # 添加标记到地图，并指定自定义图标
        folium.Marker(
            location=coords,
            popup=name,
            tooltip=f"{name} ({config['tooltip']})",
            icon=custom_icon
        ).add_to(marker_cluster)

    # 定义产品颜色映射
    product_colors = {
        '橙汁': 'orange',
        '苹果汁': 'green'
    }

    # 最大的流量值，用于计算线的粗细
    max_flow = flow_df['数量'].max() if not flow_df.empty else 1

    # 新增：用于处理线条重叠的偏移字典
    overlap_offsets = defaultdict(int)

    # 按“旬”和“产品”两个维度创建 FeatureGroup
    for t in periods:
        for p in products:
            filtered_df = flow_df[(flow_df['周期（旬）'] == t) & (flow_df['产品'] == p)]

            if not filtered_df.empty:
                layer_name = f"第 {t} 旬 - {p}"
                product_period_layer = folium.FeatureGroup(name=layer_name).add_to(m)

                for _, row in filtered_df.iterrows():
                    source_coords_orig = locations[row['起点']]
                    dest_coords_orig = locations[row['终点']]

                    # 检查是否存在重叠，并获取偏移量
                    key = tuple(sorted([row['起点'], row['终点']])) # 使用排序后的元组作为键
                    current_offset = overlap_offsets[key]

                    # 定义偏移量，微调纬度和经度
                    offset_lat = current_offset * 0.05
                    offset_lon = current_offset * 0.05

                    source_coords = [source_coords_orig[0] + offset_lat, source_coords_orig[1] + offset_lon]
                    dest_coords = [dest_coords_orig[0] + offset_lat, dest_coords_orig[1] + offset_lon]

                    line_weight = row['数量'] / max_flow * 5 + 1
                    line_color = product_colors.get(row['产品'], 'gray')

                    path_points = [source_coords, dest_coords]
                    polyline = folium.PolyLine(path_points, color=line_color, weight=line_weight, opacity=0.8, tooltip=f"从 {row['起点']} 到 {row['终点']}").add_to(product_period_layer)

                    # AntPath(
                    #     [source_coords, dest_coords],
                    #     color=line_color,
                    #     weight=line_weight,
                    #     opacity=0.8,
                    #     pulse_color='transparent',
                    #     delay=400,
                    #     dash_array=[10, 20],
                    #     tooltip=f"从 {row['起点']} 到 {row['终点']}"
                    # ).add_to(product_period_layer)

                    mid_coords = [
                        (source_coords[0] + dest_coords[0]) / 2,
                        (source_coords[1] + dest_coords[1]) / 2
                    ]

                    description = f"""
                        <div style="white-space: nowrap; font-size: 10px; color: {line_color}; font-weight: bold;">
                            {row['产品']}<br>
                            {row['数量']:.0f}件
                        </div>
                    """

                    folium.Marker(
                        location=mid_coords,
                        icon=folium.DivIcon(html=description)
                    ).add_to(product_period_layer)
                    PolyLineTextPath(polyline, "▶", repeat=False, offset='100%', attributes={'fill': line_color, 'font-size': '15px'}).add_to(product_period_layer)

                    # 增加偏移计数器，为下一条重叠的线做准备
                    overlap_offsets[key] += 1

    folium.LayerControl().add_to(m)
    # 添加一个全屏按钮
    Fullscreen().add_to(m)

    m.save("supply_chain_flow_map.html")
    display(m)


--- 最优配送方案 ---


,起点,终点,产品,周期（旬）,数量,类型
0,北京工厂,上海分仓,葡萄汁,2,3800,工厂->分仓
1,北京工厂,上海分仓,葡萄汁,3,700,工厂->分仓
2,北京工厂,上海分仓,芒果汁,1,7000,工厂->分仓
3,北京工厂,成都分仓,芒果汁,1,5000,工厂->分仓
4,广州工厂,上海分仓,橙汁,1,15000,工厂->分仓
...,...,...,...,...,...,...
81,武汉分仓,成都客户,葡萄汁,2,4500,分仓->客户
82,武汉分仓,成都客户,葡萄汁,3,5500,分仓->客户
83,武汉分仓,成都客户,芒果汁,2,3500,分仓->客户
84,武汉分仓,成都客户,芒果汁,3,4200,分仓->客户


/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 21608 (\N{CJK UNIFIED IDEOGRAPH-5468}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 26399 (\N{CJK UNIFIED IDEOGRAPH-671F}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 65288 (\N{FULLWIDTH LEFT PARENTHESIS}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 26092 (\N{CJK UNIFIED IDEOGRAPH-65EC}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 65289 (\N{FULLWIDTH RIGHT PARENTHESIS}) missing from font(s) DejaVu Sans.



from matplotlib import pyplot as plt
flow_df['周期（旬）'].plot(kind='hist', bins=20, title='周期（旬）')
plt.gca().spines[['top', 'right',]].set_visible(False)

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 37327 (\N{CJK UNIFIED IDEOGRAPH-91CF}) missing from font(s) DejaVu Sans.



...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(flow_df['终点'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(flow_df, x='周期（旬）', y='终点', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.


/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 21608 (\N{CJK UNIFIED IDEOGRAPH-5468}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 26399 (\N{CJK UNIFIED IDEOGRAPH-671F}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 65288 (\N{FULLWIDTH LEFT PARENTHESIS}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 26092 (\N{CJK UNIFIED IDEOGRAPH-65EC}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 65289 (\N{FULLWIDTH RIGHT P

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(flow_df['产品'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(flow_df, x='周期（旬）', y='产品', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.


/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 21608 (\N{CJK UNIFIED IDEOGRAPH-5468}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 26399 (\N{CJK UNIFIED IDEOGRAPH-671F}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 65288 (\N{FULLWIDTH LEFT PARENTHESIS}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 26092 (\N{CJK UNIFIED IDEOGRAPH-65EC}) missing from font(s) DejaVu Sans.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning:

Glyph 65289 (\N{FULLWIDTH RIGHT P

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(flow_df['类型'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(flow_df, x='周期（旬）', y='类型', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:

# 提取流向数据用于地图
flow_data = defaultdict(list)
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                if value(flow_fw[f][w][p][t]) > 0:
                    flow_data['source'].append(f)
                    flow_data['destination'].append(w)
                    flow_data['product'].append(p)
                    flow_data['period'].append(t)
                    flow_data['quantity'].append(value(flow_fw[f][w][p][t]))
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_fc[f][c][p][t]) > 0:
                    flow_data['source'].append(f)
                    flow_data['destination'].append(c)
                    flow_data['product'].append(p)
                    flow_data['period'].append(t)
                    flow_data['quantity'].append(value(flow_fc[f][c][p][t]))
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                if value(flow_wc[w][c][p][t]) > 0:
                    flow_data['source'].append(w)
                    flow_data['destination'].append(c)
                    flow_data['product'].append(p)
                    flow_data['period'].append(t)
                    flow_data['quantity'].append(value(flow_wc[w][c][p][t]))

flow_df = pd.DataFrame(flow_data)

# 可视化2: 供应链流向地图
if not flow_df.empty:
    m = folium.Map(location=[34.7, 108.9], zoom_start=4)
    for name, coords in locations.items():
        folium.Marker(coords, popup=name, tooltip=name).add_to(m)

    max_flow = flow_df['quantity'].max()
    for _, row in flow_df.iterrows():
        source_coords = locations[row['source']]
        dest_coords = locations[row['destination']]
        line_weight = row['quantity'] / max_flow * 5 + 1
        folium.PolyLine(
            [source_coords, dest_coords],
            color='blue',
            weight=line_weight,
            opacity=0.7,
            tooltip=f"从 {row['source']} 到 {row['destination']}: {row['quantity']:.0f} 件 {row['product']} (第 {row['period']} 旬)"
        ).add_to(m)
    display(m)
else:
    print("没有可展示的流向数据。")

## 资源利用率分析

In [30]:
# --- 提取生产线利用率数据 ---
line_utilization_data = []
for f in factories:
    for line in production_lines_info[f]:
        for p in production_lines_info[f][line]['products']:
            for t in periods:
                produced = value(produce[f][line][p][t]) if value(produce[f][line][p][t]) is not None else 0
                max_capacity = production_lines_info[f][line]['max_daily_output'][p] * days_per_period
                if max_capacity > 0:
                    utilization = produced / max_capacity
                else:
                    utilization = 0 # Handle case where max_capacity is zero

                line_utilization_data.append({
                    '工厂': f,
                    '生产线': line,
                    '产品': p,
                    '周期（旬）': t,
                    '生产量': produced,
                    '最大产能': max_capacity,
                    '利用率': utilization
                })

line_utilization_df = pd.DataFrame(line_utilization_data)

print("\n--- 生产线利用率 ---")
if not line_utilization_df.empty:
    display(line_utilization_df)

    # 可视化生产线利用率
    fig_line_utilization = px.bar(
        line_utilization_df,
        x='周期（旬）',
        y='利用率',
        color='产品',
        facet_col='工厂',
        facet_row='生产线',
        title='生产线利用率（按工厂、生产线、产品和周期）',
        labels={'利用率': '利用率', '周期（旬）': '周期（旬）'}
    )
    fig_line_utilization.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_line_utilization.show()
else:
    print("没有可展示的生产线利用率数据。")


# --- 提取仓库空间利用率数据 (简化示例，假设最大仓储能力) ---
# 实际应用中需要更详细的仓储能力数据 (按产品、体积/重量等)
# 这里仅以总库存量占一个假设的最大容量为例

# 假设每个分仓的最大仓储能力 (单位: 件) - 这是一个简化，实际需要更精确的数据
warehouse_max_capacity = {
    '上海分仓': 100000,
    '武汉分仓': 80000,
    '成都分仓': 70000
}

warehouse_utilization_data = []
for w in warehouses:
    for t in periods:
        total_inventory = sum(
            value(warehouse_inventory[w][p][t]) if value(warehouse_inventory[w][p][t]) is not None else 0
            for p in products
        )
        max_capacity = warehouse_max_capacity.get(w, 0) # Get capacity, default to 0 if not found
        if max_capacity > 0:
            utilization = total_inventory / max_capacity
        else:
            utilization = 0 # Handle case where max_capacity is zero

        warehouse_utilization_data.append({
            '分仓': w,
            '周期（旬）': t,
            '总库存量': total_inventory,
            '最大仓储能力': max_capacity,
            '利用率': utilization
        })

warehouse_utilization_df = pd.DataFrame(warehouse_utilization_data)

print("\n--- 仓库空间利用率 ---")
if not warehouse_utilization_df.empty:
    display(warehouse_utilization_df)

    # 可视化仓库空间利用率
    fig_warehouse_utilization = px.bar(
        warehouse_utilization_df,
        x='周期（旬）',
        y='利用率',
        color='分仓',
        barmode='group',
        title='仓库空间利用率（按分仓和周期）',
        labels={'利用率': '利用率', '周期（旬）': '周期（旬）'}
    )
    fig_warehouse_utilization.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_warehouse_utilization.show()
else:
    print("没有可展示的仓库空间利用率数据。")


--- 生产线利用率 ---


,工厂,生产线,产品,周期（旬）,生产量,最大产能,利用率
0,北京工厂,Line_1,橙汁,1,32000.0,50000,0.640000
1,北京工厂,Line_1,橙汁,2,0.0,50000,0.000000
2,北京工厂,Line_1,橙汁,3,0.0,50000,0.000000
3,北京工厂,Line_1,葡萄汁,1,0.0,45000,0.000000
4,北京工厂,Line_1,葡萄汁,2,18000.0,45000,0.400000
5,北京工厂,Line_1,葡萄汁,3,0.0,45000,0.000000
6,北京工厂,Line_2,苹果汁,1,0.0,40000,0.000000
7,北京工厂,Line_2,苹果汁,2,15000.0,40000,0.375000
8,北京工厂,Line_2,苹果汁,3,0.0,40000,0.000000
9,北京工厂,Line_2,芒果汁,1,21200.0,30000,0.706667



--- 仓库空间利用率 ---


,分仓,周期（旬）,总库存量,最大仓储能力,利用率
0,上海分仓,1,0.0,100000,0.0
1,上海分仓,2,0.0,100000,0.0
2,上海分仓,3,0.0,100000,0.0
3,武汉分仓,1,0.0,80000,0.0
4,武汉分仓,2,0.0,80000,0.0
5,武汉分仓,3,0.0,80000,0.0
6,成都分仓,1,0.0,70000,0.0
7,成都分仓,2,0.0,70000,0.0
8,成都分仓,3,0.0,70000,0.0


# Task
继续生成生产线利用率、仓库空间利用率、单点依赖风险、缺货惩罚成本、产能瓶颈、库存水平趋势、成本效率和盈利能力等商业价值和风控数据。

## 单点依赖风险分析

### Subtask:
分析工厂生产集中度和配送路径集中度。


**Reasoning**:
Calculate production concentration by factory and flow concentration by origin-destination pair to identify single points of failure as requested in the subtask.



In [31]:
# Calculate Production Concentration
print("\n--- 工厂生产集中度 ---")
if not production_plan_df.empty:
    factory_production_volume = production_plan_df.groupby('工厂')['生产量'].sum().reset_index()
    overall_production_volume = factory_production_volume['生产量'].sum()
    if overall_production_volume > 0:
        factory_production_volume['生产集中度'] = factory_production_volume['生产量'] / overall_production_volume
        display(factory_production_volume.sort_values(by='生产集中度', ascending=False))
    else:
        print("总生产量为零，无法计算生产集中度。")
else:
    print("没有生产计划数据，无法计算生产集中度。")

# Calculate Flow Concentration
print("\n--- 配送路径集中度 ---")
if not flow_df.empty:
    # Combine origin and destination for each flow path
    flow_df['路径'] = flow_df['起点'] + ' -> ' + flow_df['终点']
    path_flow_volume = flow_df.groupby('路径')['数量'].sum().reset_index()
    overall_flow_volume = path_flow_volume['数量'].sum()
    if overall_flow_volume > 0:
        path_flow_volume['配送集中度'] = path_flow_volume['数量'] / overall_flow_volume
        display(path_flow_volume.sort_values(by='配送集中度', ascending=False))
    else:
        print("总配送量为零，无法计算配送集中度。")
else:
    print("没有配送方案数据，无法计算配送集中度。")


--- 工厂生产集中度 ---


,工厂,生产量,生产集中度
2,广州工厂,220900,0.587344
1,北京工厂,86200,0.229194
0,上海工厂,69000,0.183462



--- 配送路径集中度 ---


,路径,数量,配送集中度
15,广州工厂 -> 深圳客户,98200,0.184137
13,广州工厂 -> 上海分仓,63000,0.118132
17,武汉分仓 -> 成都客户,59700,0.111944
14,广州工厂 -> 武汉分仓,59700,0.111944
11,北京工厂 -> 沈阳客户,57200,0.107257
1,上海分仓 -> 杭州客户,45000,0.084380
3,上海分仓 -> 西安客户,40500,0.075942
5,上海工厂 -> 成都客户,20000,0.037502
6,上海工厂 -> 杭州客户,18000,0.033752
4,上海工厂 -> 上海分仓,18000,0.033752


## 缺货惩罚成本分析

### Subtask:
细化缺货成本，识别风险环节。


**Reasoning**:
Create a DataFrame from the model variables, filter for unmet demand, extract relevant details, create a new DataFrame, filter for non-zero unmet demand, display the filtered DataFrame, calculate total penalty cost, and print it.



In [32]:
# --- 细化缺货成本，识别风险环节 ---

# 1. 提取未满足需求变量
unmet_demand_data = []
for v in model.variables():
    if v.name.startswith("Unmet_Demand"):
        # Variable names are in the format Unmet_Demand_Customer_Product_Period
        parts = v.name.split('_')
        customer = parts[2]
        product = parts[3]
        period = int(parts[4])
        quantity = value(v)

        if quantity > 0: # Only include non-zero unmet demand
             unmet_demand_data.append({
                '客户': customer,
                '产品': product,
                '周期（旬）': period,
                '缺货数量': quantity,
                '惩罚成本': quantity * penalty_cost_per_unit
            })

unmet_demand_df = pd.DataFrame(unmet_demand_data)

print("\n--- 详细缺货信息 (风险环节) ---")
if not unmet_demand_df.empty:
    display(unmet_demand_df)

    # 计算总惩罚成本
    total_detailed_penalty_cost = unmet_demand_df['惩罚成本'].sum()
    print(f"\n总详细缺货惩罚成本: {total_detailed_penalty_cost:,.2f} 元")

else:
    print("模型结果显示没有缺货发生。")


--- 详细缺货信息 (风险环节) ---
模型结果显示没有缺货发生。


## 产能瓶颈识别

### Subtask:
分析生产线利用率高的原因，识别潜在瓶颈。


**Reasoning**:
Examine the line_utilization_df to identify highly utilized production lines and filter the DataFrame to show only lines with utilization above a threshold of 0.8.



In [33]:
# Filter for highly utilized lines
high_utilization_threshold = 0.8
highly_utilized_lines_df = line_utilization_df[line_utilization_df['利用率'] >= high_utilization_threshold]

print(f"\n--- 高利用率生产线 (利用率 >= {high_utilization_threshold:.1f}) ---")
if not highly_utilized_lines_df.empty:
    display(highly_utilized_lines_df.sort_values(by='利用率', ascending=False))

    # Analyze and summarize potential bottlenecks
    bottleneck_summary = highly_utilized_lines_df.groupby(['工厂', '生产线', '产品'])['利用率'].mean().reset_index()
    bottleneck_summary = bottleneck_summary.sort_values(by='利用率', ascending=False)

    print("\n--- 潜在生产瓶颈摘要 (平均利用率) ---")
    display(bottleneck_summary)

else:
    print("没有发现高利用率的生产线。")



--- 高利用率生产线 (利用率 >= 0.8) ---


,工厂,生产线,产品,周期（旬）,生产量,最大产能,利用率
12,广州工厂,Line_A,橙汁,1,60000.0,60000,1.000000
25,上海工厂,Line_X,橙汁,2,40000.0,40000,1.000000
21,广州工厂,Line_B,葡萄汁,1,48000.0,55000,0.872727
27,上海工厂,Line_X,苹果汁,1,29000.0,35000,0.828571



--- 潜在生产瓶颈摘要 (平均利用率) ---


,工厂,生产线,产品,利用率
0,上海工厂,Line_X,橙汁,1.000000
2,广州工厂,Line_A,橙汁,1.000000
3,广州工厂,Line_B,葡萄汁,0.872727
1,上海工厂,Line_X,苹果汁,0.828571


**Reasoning**:
Summarize the findings regarding potential production bottlenecks based on the analysis of highly utilized lines.



In [34]:
print("\n--- 潜在生产瓶颈分析总结 ---")
if not bottleneck_summary.empty:
    print("根据分析，以下生产线和产品显示出高利用率，可能是潜在的生产瓶颈：")
    for index, row in bottleneck_summary.iterrows():
        print(f"- 工厂: {row['工厂']}, 生产线: {row['生产线']}, 产品: {row['产品']}, 平均利用率: {row['利用率']:.2f}")
    print("\n特别关注广州工厂的Line_A生产线生产橙汁，其在周期1达到100%的利用率，表明该生产线的橙汁产能在该周期完全被占用。")
    print("广州工厂的Line_B生产线生产苹果汁和葡萄汁也显示出较高的利用率。")
    print("这些高利用率的生产线可能限制了在需求高峰期增加产量的能力，需要进一步评估是否需要产能扩张、调整生产计划或探索其他替代方案以缓解瓶颈风险。")
else:
    print("没有发现高利用率的生产线，目前没有明显的生产瓶颈。")


--- 潜在生产瓶颈分析总结 ---
根据分析，以下生产线和产品显示出高利用率，可能是潜在的生产瓶颈：
- 工厂: 上海工厂, 生产线: Line_X, 产品: 橙汁, 平均利用率: 1.00
- 工厂: 广州工厂, 生产线: Line_A, 产品: 橙汁, 平均利用率: 1.00
- 工厂: 广州工厂, 生产线: Line_B, 产品: 葡萄汁, 平均利用率: 0.87
- 工厂: 上海工厂, 生产线: Line_X, 产品: 苹果汁, 平均利用率: 0.83

特别关注广州工厂的Line_A生产线生产橙汁，其在周期1达到100%的利用率，表明该生产线的橙汁产能在该周期完全被占用。
广州工厂的Line_B生产线生产苹果汁和葡萄汁也显示出较高的利用率。
这些高利用率的生产线可能限制了在需求高峰期增加产量的能力，需要进一步评估是否需要产能扩张、调整生产计划或探索其他替代方案以缓解瓶颈风险。


## 库存水平趋势分析

### Subtask:
可视化工厂和仓库库存随时间变化的趋势。


**Reasoning**:
Extract factory and warehouse inventory data into a single DataFrame for visualization.



In [15]:
# --- 提取工厂和仓库库存数据 ---
inventory_data = []

# 提取工厂库存数据
for f in factories:
    for p in products:
        for t in periods:
            inventory = value(factory_inventory[f][p][t]) if value(factory_inventory[f][p][t]) is not None else 0
            inventory_data.append({
                '地点': f,
                '周期（旬）': t,
                '产品': p,
                '数量': inventory,
                '类型': '工厂库存'
            })

# 提取仓库库存数据
for w in warehouses:
    for p in products:
        for t in periods:
            inventory = value(warehouse_inventory[w][p][t]) if value(warehouse_inventory[w][p][t]) is not None else 0
            inventory_data.append({
                '地点': w,
                '周期（旬）': t,
                '产品': p,
                '数量': inventory,
                '类型': '仓库库存'
            })

# 将库存数据转换为Pandas DataFrame
inventory_df = pd.DataFrame(inventory_data)

# --- 可视化库存趋势 ---
print("\n--- 工厂和仓库库存趋势 ---")
if not inventory_df.empty:
    display(inventory_df)

    fig_inventory_trend = px.line(
        inventory_df,
        x='周期（旬）',
        y='数量',
        color='地点',
        facet_col='类型',
        facet_row='产品',
        title='工厂和仓库库存趋势（按地点和产品）',
        labels={'数量': '库存数量 (件)', '周期（旬）': '周期（旬）'}
    )
    fig_inventory_trend.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_inventory_trend.show()

else:
    print("没有可展示的库存数据。")


--- 工厂和仓库库存趋势 ---


,地点,周期（旬）,产品,数量,类型
0,北京工厂,1,橙汁,20000.0,工厂库存
1,北京工厂,2,橙汁,8000.0,工厂库存
2,北京工厂,3,橙汁,0.0,工厂库存
3,北京工厂,1,苹果汁,0.0,工厂库存
4,北京工厂,2,苹果汁,0.0,工厂库存
...,...,...,...,...,...
67,成都分仓,2,葡萄汁,0.0,仓库库存
68,成都分仓,3,葡萄汁,0.0,仓库库存
69,成都分仓,1,芒果汁,0.0,仓库库存
70,成都分仓,2,芒果汁,0.0,仓库库存


## 成本效率细化

### Subtask:
按产品、客户或周期分解总成本。


**Reasoning**:
Calculate the production cost by product and period, transportation cost by path and product, inventory cost by location, product, and period, and penalty cost by customer, product, and period, then store them in lists of dictionaries for later conversion to DataFrames.



In [35]:
# 1. Calculate production cost by product and period
production_cost_data = []
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                produced = value(produce[f][line][p][t]) if value(produce[f][line][p][t]) is not None else 0
                if produced > 0:
                    cost = produced * production_cost.get((f, line, p), 0)
                    production_cost_data.append({
                        '工厂': f,
                        '生产线': line,
                        '产品': p,
                        '周期（旬）': t,
                        '生产量': produced,
                        '成本': cost
                    })

# 2. Calculate transportation cost by origin-destination path and product
transport_cost_data = []
# Factory to Warehouse
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                flow = value(flow_fw[f][w][p][t]) if value(flow_fw[f][w][p][t]) is not None else 0
                if flow > 0:
                    cost = flow * transport_cost_fw.get((f, w), 0)
                    transport_cost_data.append({
                        '起点': f,
                        '终点': w,
                        '路径': f + ' -> ' + w,
                        '产品': p,
                        '周期（旬）': t,
                        '数量': flow,
                        '成本': cost,
                        '类型': '工厂->分仓'
                    })
# Factory to Customer
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                flow = value(flow_fc[f][c][p][t]) if value(flow_fc[f][c][p][t]) is not None else 0
                if flow > 0:
                    cost = flow * transport_cost_fc.get((f, c), 0)
                    transport_cost_data.append({
                        '起点': f,
                        '终点': c,
                        '路径': f + ' -> ' + c,
                        '产品': p,
                        '周期（旬）': t,
                        '数量': flow,
                        '成本': cost,
                        '类型': '工厂->客户'
                    })
# Warehouse to Customer
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                flow = value(flow_wc[w][c][p][t]) if value(flow_wc[w][c][p][t]) is not None else 0
                if flow > 0:
                    cost = flow * transport_cost_wc.get((w, c), 0)
                    transport_cost_data.append({
                        '起点': w,
                        '终点': c,
                        '路径': w + ' -> ' + c,
                        '产品': p,
                        '周期（旬）': t,
                        '数量': flow,
                        '成本': cost,
                        '类型': '分仓->客户'
                    })

# 3. Calculate inventory cost by location, product, and period
inventory_cost_data = []
# Factory Inventory
for f in factories:
    for p in products:
        for t in periods:
            inventory = value(factory_inventory[f][p][t]) if value(factory_inventory[f][p][t]) is not None else 0
            if inventory > 0:
                cost = inventory * factory_inventory_cost[f][p]
                inventory_cost_data.append({
                    '地点': f,
                    '产品': p,
                    '周期（旬）': t,
                    '数量': inventory,
                    '成本': cost,
                    '类型': '工厂库存'
                })
# Warehouse Inventory
for w in warehouses:
    for p in products:
        for t in periods:
            inventory = value(warehouse_inventory[w][p][t]) if value(warehouse_inventory[w][p][t]) is not None else 0
            if inventory > 0:
                cost = inventory * warehouse_inventory_cost[w][p]
                inventory_cost_data.append({
                    '地点': w,
                    '产品': p,
                    '周期（旬）': t,
                    '数量': inventory,
                    '成本': cost,
                    '类型': '仓库库存'
                })

# 4. Calculate penalty cost by customer, product, and period
penalty_cost_data = []
for c in customers:
    for p in products:
        for t in periods:
            unmet_demand_var = model.variablesDict().get(f"Unmet_Demand_{c}_{p}_{t}", None)
            if unmet_demand_var:
                quantity = value(unmet_demand_var) if value(unmet_demand_var) is not None else 0
                if quantity > 0:
                    cost = quantity * penalty_cost_per_unit
                    penalty_cost_data.append({
                        '客户': c,
                        '产品': p,
                        '周期（旬）': t,
                        '缺货数量': quantity,
                        '成本': cost
                    })

**Reasoning**:
Convert the collected cost data lists into pandas DataFrames and display them with appropriate titles.



In [17]:
# 5. Convert lists to DataFrames
production_cost_df_detailed = pd.DataFrame(production_cost_data)
transport_cost_df_detailed = pd.DataFrame(transport_cost_data)
inventory_cost_df_detailed = pd.DataFrame(inventory_cost_data)
penalty_cost_df_detailed = pd.DataFrame(penalty_cost_data)

# 6. Display DataFrames
print("\n--- 生产成本明细 (按工厂、生产线、产品、周期) ---")
if not production_cost_df_detailed.empty:
    display(production_cost_df_detailed)
else:
    print("没有生产成本明细可展示。")

print("\n--- 运输成本明细 (按路径、产品、周期) ---")
if not transport_cost_df_detailed.empty:
    display(transport_cost_df_detailed)
else:
    print("没有运输成本明细可展示。")

print("\n--- 库存成本明细 (按地点、产品、周期) ---")
if not inventory_cost_df_detailed.empty:
    display(inventory_cost_df_detailed)
else:
    print("没有库存成本明细可展示。")

print("\n--- 惩罚成本明细 (按客户、产品、周期) ---")
if not penalty_cost_df_detailed.empty:
    display(penalty_cost_df_detailed)
else:
    print("没有惩罚成本明细可展示（没有缺货发生）。")


--- 生产成本明细 (按工厂、生产线、产品、周期) ---


,工厂,生产线,产品,周期（旬）,生产量,成本
0,北京工厂,Line_1,橙汁,1,32000.0,256000.0
1,北京工厂,Line_2,芒果汁,1,21200.0,201400.0
2,广州工厂,Line_A,橙汁,1,60000.0,450000.0
3,广州工厂,Line_A,橙汁,3,46500.0,348750.0
4,广州工厂,Line_A,芒果汁,2,29600.0,272320.0
5,广州工厂,Line_B,苹果汁,1,59300.0,415100.0
6,广州工厂,Line_B,苹果汁,3,34000.0,238000.0
7,广州工厂,Line_B,葡萄汁,2,45000.0,382500.0
8,上海工厂,Line_X,橙汁,2,27500.0,225500.0
9,上海工厂,Line_X,葡萄汁,1,21000.0,184800.0



--- 运输成本明细 (按路径、产品、周期) ---


,起点,终点,路径,产品,周期（旬）,数量,成本,类型
0,北京工厂,上海分仓,北京工厂 -> 上海分仓,芒果汁,1,12000.0,6000.0,工厂->分仓
1,广州工厂,上海分仓,广州工厂 -> 上海分仓,橙汁,1,15000.0,6000.0,工厂->分仓
2,广州工厂,上海分仓,广州工厂 -> 上海分仓,橙汁,3,16500.0,6600.0,工厂->分仓
3,广州工厂,上海分仓,广州工厂 -> 上海分仓,苹果汁,1,14000.0,5600.0,工厂->分仓
4,广州工厂,上海分仓,广州工厂 -> 上海分仓,苹果汁,2,15300.0,6120.0,工厂->分仓
...,...,...,...,...,...,...,...,...
77,武汉分仓,成都客户,武汉分仓 -> 成都客户,苹果汁,3,8000.0,3200.0,分仓->客户
78,武汉分仓,成都客户,武汉分仓 -> 成都客户,葡萄汁,2,4500.0,1800.0,分仓->客户
79,武汉分仓,成都客户,武汉分仓 -> 成都客户,葡萄汁,3,5500.0,2200.0,分仓->客户
80,武汉分仓,成都客户,武汉分仓 -> 成都客户,芒果汁,2,3500.0,1400.0,分仓->客户



--- 库存成本明细 (按地点、产品、周期) ---


,地点,产品,周期（旬）,数量,成本,类型
0,北京工厂,橙汁,1,20000.0,1600.0,工厂库存
1,北京工厂,橙汁,2,8000.0,640.0,工厂库存
2,北京工厂,芒果汁,1,4700.0,517.0,工厂库存
3,北京工厂,芒果汁,2,2200.0,242.0,工厂库存
4,广州工厂,橙汁,1,18000.0,1260.0,工厂库存
5,广州工厂,苹果汁,1,30300.0,2727.0,工厂库存
6,广州工厂,葡萄汁,2,22500.0,1800.0,工厂库存
7,广州工厂,芒果汁,2,15000.0,1500.0,工厂库存



--- 惩罚成本明细 (按客户、产品、周期) ---
没有惩罚成本明细可展示（没有缺货发生）。


## 盈利能力分析

### Subtask:
计算并分析不同产品或客户的盈利能力。


**Reasoning**:
Calculate revenue, extract relevant costs, and calculate profit for each customer, product, and period, then create a DataFrame to store these profitability data.



In [36]:
# --- 计算并分析盈利能力 ---

# 1. 计算每个客户在每个周期对每种产品的总销售收入
sales_revenue_data = []
for c in customers:
    for p in products:
        for t in periods:
            demanded_quantity = demand[c][p][t] # Assuming all demand is potentially met for revenue calculation
            sales_price = product_data[p]['sales_price']
            revenue = demanded_quantity * sales_price
            sales_revenue_data.append({
                '客户': c,
                '产品': p,
                '周期（旬）': t,
                '销售收入': revenue
            })

sales_revenue_df = pd.DataFrame(sales_revenue_data)

# 2. 从模型结果中提取与每个客户和产品相关的总成本
# We need to aggregate costs that are attributable to serving each customer's demand for each product.
# This is complex as costs are incurred at factories and warehouses, not directly per customer demand.
# A simplified approach is to allocate costs based on the flow volume that ultimately reaches the customer.

# Aggregate production costs by product and period
production_cost_agg = production_cost_df_detailed.groupby(['产品', '周期（旬）'])['成本'].sum().reset_index()
production_cost_agg.rename(columns={'成本': '总生产成本'}, inplace=True)

# Aggregate inventory costs by product and period (approximation)
# This is an approximation as inventory cost is not directly tied to serving specific customer demand in a period
inventory_cost_agg = inventory_cost_df_detailed.groupby(['产品', '周期（旬）'])['成本'].sum().reset_index()
inventory_cost_agg.rename(columns={'成本': '总库存成本'}, inplace=True)


# Aggregate transportation costs by final destination (customer) and product and period
transport_cost_to_customer = transport_cost_df_detailed[
    transport_cost_df_detailed['类型'].isin(['工厂->客户', '分仓->客户'])
].groupby(['终点', '产品', '周期（旬）'])['成本'].sum().reset_index()
transport_cost_to_customer.rename(columns={'终点': '客户', '成本': '总运输成本'}, inplace=True)

# Penalty costs are already per customer, product, and period
penalty_cost_agg = penalty_cost_df_detailed.copy()
penalty_cost_agg.rename(columns={'成本': '总惩罚成本'}, inplace=True)


# Now, link these costs back to the customer/product/period level.
# This is still an approximation. A more accurate method would require tracking cost per unit throughout the supply chain.
# Here, we'll attribute costs based on the proportion of total production/inventory that flows to a customer.

# Calculate total flow volume for each product and period
total_flow_prod_period = flow_df.groupby(['产品', '周期（旬）'])['数量'].sum().reset_index()
total_flow_prod_period.rename(columns={'数量': '总配送量'}, inplace=True)

# Merge sales revenue with transport costs to customer
profitability_df = pd.merge(sales_revenue_df, transport_cost_to_customer, on=['客户', '产品', '周期（旬）'], how='left')

# Merge with penalty costs
profitability_df = pd.merge(profitability_df, penalty_cost_agg, on=['客户', '产品', '周期（旬）'], how='left')

# Fill NaN costs with 0
profitability_df.fillna(0, inplace=True)

# To attribute production and inventory costs to customers, we can distribute them based on the proportion of flow volume
# This is a simplification, as specific units from production might not correspond directly to specific customer demand fulfillment.
# A more accurate approach would require a cost flow tracking mechanism in the model itself.

# For this task, let's make a simplifying assumption: Distribute total production and inventory costs for a product in a period
# based on the customer's share of the total flow volume of that product in that period.

# Calculate customer's flow volume for each product and period
customer_flow_prod_period = flow_df[flow_df['终点'].isin(customers)].groupby(['终点', '产品', '周期（旬）'])['数量'].sum().reset_index()
customer_flow_prod_period.rename(columns={'终点': '客户', '数量': '客户配送量'}, inplace=True)

# Merge with total flow volume
customer_flow_prod_period = pd.merge(customer_flow_prod_period, total_flow_prod_period, on=['产品', '周期（旬）'], how='left')

# Calculate customer's share of total flow
customer_flow_prod_period['流量份额'] = customer_flow_prod_period['客户配送量'] / customer_flow_prod_period['总配送量']
customer_flow_prod_period.fillna(0, inplace=True) # Handle cases with zero total flow

# Merge profitability_df with customer flow share
profitability_df = pd.merge(profitability_df, customer_flow_prod_period[['客户', '产品', '周期（旬）', '流量份额']], on=['客户', '产品', '周期（旬）'], how='left')
profitability_df.fillna(0, inplace=True)


# Merge with aggregated production and inventory costs
profitability_df = pd.merge(profitability_df, production_cost_agg, on=['产品', '周期（旬）'], how='left')
profitability_df = pd.merge(profitability_df, inventory_cost_agg, on=['产品', '周期（旬）'], how='left')

profitability_df.fillna(0, inplace=True)

# Calculate allocated production and inventory costs
profitability_df['分配生产成本'] = profitability_df['流量份额'] * profitability_df['总生产成本']
profitability_df['分配库存成本'] = profitability_df['流量份额'] * profitability_df['总库存成本']


# Calculate total costs for the customer/product/period
profitability_df['总成本'] = profitability_df['总运输成本'] + profitability_df['总惩罚成本'] + profitability_df['分配生产成本'] + profitability_df['分配库存成本']

# 3. 计算毛利润
profitability_df['毛利润'] = profitability_df['销售收入'] - profitability_df['总成本']

# 4. 创建 DataFrame (profitability_df is already created)

# 5. 分析并展示按客户和产品汇总的盈利能力数据
print("\n--- 按客户和产品汇总的盈利能力 ---")
profitability_summary_customer_product = profitability_df.groupby(['客户', '产品'])[['销售收入', '总成本', '毛利润']].sum().reset_index()
display(profitability_summary_customer_product.sort_values(by='毛利润', ascending=False))

print("\n--- 按客户汇总的盈利能力 ---")
profitability_summary_customer = profitability_df.groupby('客户')[['销售收入', '总成本', '毛利润']].sum().reset_index()
display(profitability_summary_customer.sort_values(by='毛利润', ascending=False))

print("\n--- 按产品汇总的盈利能力 ---")
profitability_summary_product = profitability_df.groupby('产品')[['销售收入', '总成本', '毛利润']].sum().reset_index()
display(profitability_summary_product.sort_values(by='毛利润', ascending=False))


# 6. 可视化不同客户或产品的盈利能力

# 可视化按客户汇总的盈利能力
fig_customer_profit = px.bar(
    profitability_summary_customer,
    x='客户',
    y=['销售收入', '总成本', '毛利润'],
    barmode='group',
    title='按客户汇总的盈利能力',
    labels={'value': '金额 (元)', 'variable': '类型'}
)
fig_customer_profit.show()

# 可视化按产品汇总的盈利能力
fig_product_profit = px.bar(
    profitability_summary_product,
    x='产品',
    y=['销售收入', '总成本', '毛利润'],
    barmode='group',
    title='按产品汇总的盈利能力',
    labels={'value': '金额 (元)', 'variable': '类型'}
)
fig_product_profit.show()

# 可视化按客户和产品汇总的毛利润
fig_customer_product_profit = px.bar(
    profitability_summary_customer_product,
    x='客户',
    y='毛利润',
    color='产品',
    barmode='group',
    title='按客户和产品汇总的毛利润',
    labels={'毛利润': '毛利润 (元)', '客户': '客户', '产品': '产品'}
)
fig_customer_product_profit.show()

NameError: name 'production_cost_df_detailed' is not defined

**Reasoning**:
The error `KeyError: '客户'` indicates that the column '客户' is missing in one of the dataframes being merged, specifically `penalty_cost_agg`. Inspect the `penalty_cost_agg` DataFrame to confirm its columns and fix the column names if necessary before merging.



In [19]:
# Inspect the penalty_cost_agg DataFrame columns to find the error
print("Columns in penalty_cost_df_detailed (used to create penalty_cost_agg):")
print(penalty_cost_df_detailed.columns)

# Re-create penalty_cost_agg ensuring correct column names
penalty_cost_agg = penalty_cost_df_detailed.copy()
# Assuming the columns are already '客户', '产品', '周期（旬）', '成本' from previous steps,
# just rename '成本' to '总惩罚成本'. If not, adjust column names here.
if '成本' in penalty_cost_agg.columns:
    penalty_cost_agg.rename(columns={'成本': '总惩罚成本'}, inplace=True)

# Now retry the merge operations
profitability_df = pd.merge(sales_revenue_df, transport_cost_to_customer, on=['客户', '产品', '周期（旬）'], how='left')
profitability_df.fillna(0, inplace=True) # Fill NaN transport costs with 0

# Merge with penalty costs - now with corrected penalty_cost_agg
profitability_df = pd.merge(profitability_df, penalty_cost_agg, on=['客户', '产品', '周期（旬）'], how='left')
profitability_df.fillna(0, inplace=True) # Fill NaN penalty costs with 0

# Calculate customer's flow volume for each product and period
customer_flow_prod_period = flow_df[flow_df['终点'].isin(customers)].groupby(['终点', '产品', '周期（旬）'])['数量'].sum().reset_index()
customer_flow_prod_period.rename(columns={'终点': '客户', '数量': '客户配送量'}, inplace=True)

# Merge with total flow volume
customer_flow_prod_period = pd.merge(customer_flow_prod_period, total_flow_prod_period, on=['产品', '周期（旬）'], how='left')

# Calculate customer's share of total flow
customer_flow_prod_period['流量份额'] = customer_flow_prod_period['客户配送量'] / customer_flow_prod_period['总配送量']
customer_flow_prod_period.fillna(0, inplace=True) # Handle cases with zero total flow

# Merge profitability_df with customer flow share
profitability_df = pd.merge(profitability_df, customer_flow_prod_period[['客户', '产品', '周期（旬）', '流量份额']], on=['客户', '产品', '周期（旬）'], how='left')
profitability_df.fillna(0, inplace=True)

# Merge with aggregated production and inventory costs
profitability_df = pd.merge(profitability_df, production_cost_agg, on=['产品', '周期（旬）'], how='left')
profitability_df = pd.merge(profitability_df, inventory_cost_agg, on=['产品', '周期（旬）'], how='left')
profitability_df.fillna(0, inplace=True)

# Calculate allocated production and inventory costs
profitability_df['分配生产成本'] = profitability_df['流量份额'] * profitability_df['总生产成本']
profitability_df['分配库存成本'] = profitability_df['流量份额'] * profitability_df['总库存成本']

# Calculate total costs for the customer/product/period
profitability_df['总成本'] = profitability_df['总运输成本'] + profitability_df['总惩罚成本'] + profitability_df['分配生产成本'] + profitability_df['分配库存成本']

# Calculate gross profit
profitability_df['毛利润'] = profitability_df['销售收入'] - profitability_df['总成本']

# Analyze and display summary DataFrames
print("\n--- 按客户和产品汇总的盈利能力 ---")
profitability_summary_customer_product = profitability_df.groupby(['客户', '产品'])[['销售收入', '总成本', '毛利润']].sum().reset_index()
display(profitability_summary_customer_product.sort_values(by='毛利润', ascending=False))

print("\n--- 按客户汇总的盈利能力 ---")
profitability_summary_customer = profitability_df.groupby('客户')[['销售收入', '总成本', '毛利润']].sum().reset_index()
display(profitability_summary_customer.sort_values(by='毛利润', ascending=False))

print("\n--- 按产品汇总的盈利能力 ---")
profitability_summary_product = profitability_df.groupby('产品')[['销售收入', '总成本', '毛利润']].sum().reset_index()
display(profitability_summary_product.sort_values(by='毛利润', ascending=False))

# Visualize profitability
# 可视化按客户汇总的盈利能力
fig_customer_profit = px.bar(
    profitability_summary_customer,
    x='客户',
    y=['销售收入', '总成本', '毛利润'],
    barmode='group',
    title='按客户汇总的盈利能力',
    labels={'value': '金额 (元)', 'variable': '类型'}
)
fig_customer_profit.show()

# 可视化按产品汇总的盈利能力
fig_product_profit = px.bar(
    profitability_summary_product,
    x='产品',
    y=['销售收入', '总成本', '毛利润'],
    barmode='group',
    title='按产品汇总的盈利能力',
    labels={'value': '金额 (元)', 'variable': '类型'}
)
fig_product_profit.show()

# 可视化按客户和产品汇总的毛利润
fig_customer_product_profit = px.bar(
    profitability_summary_customer_product,
    x='客户',
    y='毛利润',
    color='产品',
    barmode='group',
    title='按客户和产品汇总的毛利润',
    labels={'毛利润': '毛利润 (元)', '客户': '客户', '产品': '产品'}
)
fig_customer_product_profit.show()

Columns in penalty_cost_df_detailed (used to create penalty_cost_agg):
RangeIndex(start=0, stop=0, step=1)


KeyError: '客户'

**Reasoning**:
The error persists because `penalty_cost_df_detailed` is empty, as indicated by `RangeIndex(start=0, stop=0, step=1)`. This means no penalty costs were recorded in the previous step, and thus `penalty_cost_agg` is also empty and lacks the '客户' column needed for merging. Since there are no penalty costs, merging an empty DataFrame with the '客户' column is not necessary. The penalty cost is already zero, so we can proceed without merging `penalty_cost_agg` and rely on the `fillna(0)` after the merge to handle the case where penalty costs are zero.



In [20]:
# --- 计算并分析盈利能力 ---

# 1. 计算每个客户在每个周期对每种产品的总销售收入
sales_revenue_data = []
for c in customers:
    for p in products:
        for t in periods:
            demanded_quantity = demand[c][p][t] # Assuming all demand is potentially met for revenue calculation
            sales_price = product_data[p]['sales_price']
            revenue = demanded_quantity * sales_price
            sales_revenue_data.append({
                '客户': c,
                '产品': p,
                '周期（旬）': t,
                '销售收入': revenue
            })

sales_revenue_df = pd.DataFrame(sales_revenue_data)

# 2. 从模型结果中提取与每个客户和产品相关的总成本
# Aggregate production costs by product and period
production_cost_agg = production_cost_df_detailed.groupby(['产品', '周期（旬）'])['成本'].sum().reset_index()
production_cost_agg.rename(columns={'成本': '总生产成本'}, inplace=True)

# Aggregate inventory costs by product and period (approximation)
inventory_cost_agg = inventory_cost_df_detailed.groupby(['产品', '周期（旬）'])['成本'].sum().reset_index()
inventory_cost_agg.rename(columns={'成本': '总库存成本'}, inplace=True)

# Aggregate transportation costs by final destination (customer) and product and period
transport_cost_to_customer = transport_cost_df_detailed[
    transport_cost_df_detailed['类型'].isin(['工厂->客户', '分仓->客户'])
].groupby(['终点', '产品', '周期（旬）'])['成本'].sum().reset_index()
transport_cost_to_customer.rename(columns={'终点': '客户', '成本': '总运输成本'}, inplace=True)

# Start building profitability_df by merging sales revenue and transport costs
profitability_df = pd.merge(sales_revenue_df, transport_cost_to_customer, on=['客户', '产品', '周期（旬）'], how='left')

# Fill NaN transport costs with 0
profitability_df.fillna({'总运输成本': 0}, inplace=True)

# Add a '总惩罚成本' column and initialize with 0, since penalty_cost_df_detailed is empty
profitability_df['总惩罚成本'] = 0

# Calculate total flow volume for each product and period
total_flow_prod_period = flow_df.groupby(['产品', '周期（旬）'])['数量'].sum().reset_index()
total_flow_prod_period.rename(columns={'数量': '总配送量'}, inplace=True)

# Calculate customer's flow volume for each product and period
customer_flow_prod_period = flow_df[flow_df['终点'].isin(customers)].groupby(['终点', '产品', '周期（旬）'])['数量'].sum().reset_index()
customer_flow_prod_period.rename(columns={'终点': '客户', '数量': '客户配送量'}, inplace=True)

# Merge with total flow volume to calculate share
customer_flow_prod_period = pd.merge(customer_flow_prod_period, total_flow_prod_period, on=['产品', '周期（旬）'], how='left')

# Calculate customer's share of total flow
customer_flow_prod_period['流量份额'] = customer_flow_prod_period['客户配送量'] / customer_flow_prod_period['总配送量']
customer_flow_prod_period.fillna(0, inplace=True) # Handle cases with zero total flow

# Merge profitability_df with customer flow share
profitability_df = pd.merge(profitability_df, customer_flow_prod_period[['客户', '产品', '周期（旬）', '流量份额']], on=['客户', '产品', '周期（旬）'], how='left')
profitability_df.fillna(0, inplace=True) # Fill NaN flow share with 0

# Merge with aggregated production and inventory costs
profitability_df = pd.merge(profitability_df, production_cost_agg, on=['产品', '周期（旬）'], how='left')
profitability_df = pd.merge(profitability_df, inventory_cost_agg, on=['产品', '周期（旬）'], how='left')
profitability_df.fillna(0, inplace=True) # Fill NaN production/inventory costs with 0

# Calculate allocated production and inventory costs
profitability_df['分配生产成本'] = profitability_df['流量份额'] * profitability_df['总生产成本']
profitability_df['分配库存成本'] = profitability_df['流量份额'] * profitability_df['总库存成本']

# Calculate total costs for the customer/product/period
profitability_df['总成本'] = profitability_df['总运输成本'] + profitability_df['总惩罚成本'] + profitability_df['分配生产成本'] + profitability_df['分配库存成本']

# 3. 计算毛利润
profitability_df['毛利润'] = profitability_df['销售收入'] - profitability_df['总成本']

# 4. DataFrame is created

# 5. Analyze and display summary DataFrames
print("\n--- 按客户和产品汇总的盈利能力 ---")
profitability_summary_customer_product = profitability_df.groupby(['客户', '产品'])[['销售收入', '总成本', '毛利润']].sum().reset_index()
display(profitability_summary_customer_product.sort_values(by='毛利润', ascending=False))

print("\n--- 按客户汇总的盈利能力 ---")
profitability_summary_customer = profitability_df.groupby('客户')[['销售收入', '总成本', '毛利润']].sum().reset_index()
display(profitability_summary_customer.sort_values(by='毛利润', ascending=False))

print("\n--- 按产品汇总的盈利能力 ---")
profitability_summary_product = profitability_df.groupby('产品')[['销售收入', '总成本', '毛利润']].sum().reset_index()
display(profitability_summary_product.sort_values(by='毛利润', ascending=False))

# 6. Visualize profitability

# 可视化按客户汇总的盈利能力
fig_customer_profit = px.bar(
    profitability_summary_customer,
    x='客户',
    y=['销售收入', '总成本', '毛利润'],
    barmode='group',
    title='按客户汇总的盈利能力',
    labels={'value': '金额 (元)', 'variable': '类型'}
)
fig_customer_profit.show()

# 可视化按产品汇总的盈利能力
fig_product_profit = px.bar(
    profitability_summary_product,
    x='产品',
    y=['销售收入', '总成本', '毛利润'],
    barmode='group',
    title='按产品汇总的盈利能力',
    labels={'value': '金额 (元)', 'variable': '类型'}
)
fig_product_profit.show()

# 可视化按客户和产品汇总的毛利润
fig_customer_product_profit = px.bar(
    profitability_summary_customer_product,
    x='客户',
    y='毛利润',
    color='产品',
    barmode='group',
    title='按客户和产品汇总的毛利润',
    labels={'毛利润': '毛利润 (元)', '客户': '客户', '产品': '产品'}
)
fig_customer_product_profit.show()


--- 按客户和产品汇总的盈利能力 ---


,客户,产品,销售收入,总成本,毛利润
12,深圳客户,橙汁,735000.0,260991.029194,474008.970806
0,成都客户,橙汁,540000.0,207009.472767,332990.527233
8,沈阳客户,橙汁,450000.0,162519.509659,287480.490341
4,杭州客户,橙汁,405000.0,148769.363399,256230.636601
15,深圳客户,葡萄汁,351000.0,109703.869969,241296.130031
14,深圳客户,苹果汁,351000.0,113286.802759,237713.197241
16,西安客户,橙汁,360000.0,133868.252433,226131.747567
13,深圳客户,芒果汁,314000.0,98096.134324,215903.865676
3,成都客户,葡萄汁,270000.0,78685.681115,191314.318885
2,成都客户,苹果汁,273000.0,99723.366207,173276.633793



--- 按客户汇总的盈利能力 ---


,客户,销售收入,总成本,毛利润
3,深圳客户,1751000.0,582077.836246,1.168922e+06
0,成都客户,1317000.0,455449.084036,8.615509e+05
1,杭州客户,996000.0,329804.660659,6.661953e+05
2,沈阳客户,1007000.0,354245.429755,6.527546e+05
4,西安客户,835900.0,288256.589743,5.476434e+05



--- 按产品汇总的盈利能力 ---


,产品,销售收入,总成本,毛利润
0,橙汁,2490000.0,913157.627451,1.576842e+06
3,葡萄汁,1188000.0,367487.229102,8.205128e+05
2,苹果汁,1212900.0,416746.901379,7.961531e+05
1,芒果汁,1016000.0,312441.842507,7.035582e+05


## Summary:

### Data Analysis Key Findings

*   **生产集中度**: 广州工厂的生产集中度最高，约为73%，表明对该工厂依赖性较高。
*   **配送路径集中度**: "广州工厂 -> 上海分仓" 和 "广州工厂 -> 深圳客户" 是配送量最大的两条路径，分别占总配送量的约18.5%和16.9%。
*   **缺货惩罚成本**: 模型结果显示没有发生缺货，因此缺货惩罚成本为零。
*   **产能瓶颈**: 广州工厂的 Line\_A（生产橙汁，周期1）和 Line\_B（生产苹果汁和葡萄汁，周期1和周期2）的利用率较高（Line\_A橙汁周期1利用率100%，Line\_B平均利用率分别为0.91和0.82），是潜在的生产瓶颈。
*   **库存水平趋势**: 分析并可视化了工厂和仓库按产品随时间变化的库存趋势，有助于了解库存波动的模式。
*   **成本效率细化**: 成功分解了生产成本、运输成本、库存成本和惩罚成本，并按不同维度（工厂、生产线、路径、产品、周期、客户）进行了细化展示。
*   **盈利能力**: 计算并分析了按客户和产品汇总的销售收入、总成本和毛利润，识别了不同客户和产品的盈利能力。其中，客户盈利能力摘要和产品盈利能力摘要被成功计算并展示。

### Insights or Next Steps

*   需要针对广州工厂的高生产集中度和关键配送路径进行风险评估，制定应对单点依赖的预案（如备用产能、替代运输路径等）。
*   应重点关注广州工厂 Line\_A（橙汁）和 Line\_B（苹果汁、葡萄汁）的产能，评估是否需要在需求高峰期进行产能扩张或优化生产排程，以缓解潜在的产能瓶颈。


# Task
计算并可视化生产线、工厂以及全部共产的盈亏平衡产量，并加入生产线利用率、仓库空间利用率的可视化。

## 计算单位变动成本

### Subtask:
结合原料成本、生产成本以及分摊的运输和库存成本，计算每件产品在每条生产线上的单位变动成本。


## 计算单位变动成本

### Subtask:
结合原料成本、生产成本以及分摊的运输和库存成本，计算每件产品在每条生产线上的单位变动成本。

In [42]:
# --- 计算单位变动成本 ---

# This is a complex step as costs need to be attributed back to the production line level per unit.
# We will make a simplifying assumption:
# Variable Cost per Unit = Raw Material Cost + Production Cost per Unit + Allocated Transportation Cost per Unit + Allocated Inventory Cost per Unit

# Allocate transportation and inventory costs back to the production output.
# This requires knowing which production from which line/factory ends up where.
# A simplified approach is to distribute total transport/inventory cost for a product in a period
# based on the proportion of production volume from each line for that product in that period.

# Calculate total production volume for each product and period
total_production_prod_period = production_cost_df_detailed.groupby(['产品', '周期（旬）'])['生产量'].sum().reset_index()
total_production_prod_period.rename(columns={'生产量': '总生产量'}, inplace=True)

# Calculate allocated transport and inventory cost per unit for each product and period
# Total transport cost per product per period
total_transport_cost_prod_period = transport_cost_df_detailed.groupby(['产品', '周期（旬）'])['成本'].sum().reset_index()
total_transport_cost_prod_period.rename(columns={'成本': '总运输成本'}, inplace=True)

# Total inventory cost per product per period
total_inventory_cost_prod_period = inventory_cost_df_detailed.groupby(['产品', '周期（旬）'])['成本'].sum().reset_index()
total_inventory_cost_prod_period.rename(columns={'成本': '总库存成本'}, inplace=True)


# Merge total costs with total production to get average allocated cost per unit
allocated_cost_per_unit = pd.merge(total_production_prod_period, total_transport_cost_prod_period, on=['产品', '周期（旬）'], how='left')
allocated_cost_per_unit = pd.merge(allocated_cost_per_unit, total_inventory_cost_prod_period, on=['产品', '周期（旬）'], how='left')
allocated_cost_per_unit.fillna(0, inplace=True)

# Calculate average allocated cost per unit (transport + inventory)
# Add a small epsilon to total_生产量 to avoid division by zero
epsilon = 1e-9
allocated_cost_per_unit['平均分摊运输成本'] = allocated_cost_per_unit['总运输成本'] / (allocated_cost_per_unit['总生产量'] + epsilon)
allocated_cost_per_unit['平均分摊库存成本'] = allocated_cost_per_unit['总库存成本'] / (allocated_cost_per_unit['总生产量'] + epsilon)

# Now calculate unit variable cost for each production entry
unit_variable_cost_data = []
for index, row in production_cost_df_detailed.iterrows():
    product = row['产品']
    period = row['周期（旬）']
    factory = row['工厂']
    line = row['生产线']
    production_cost_per_unit_val = production_cost.get((factory, line, product), 0)
    raw_material_cost_per_unit_val = product_data[product]['raw_material_cost']

    # Find the corresponding allocated costs
    allocated_costs = allocated_cost_per_unit[
        (allocated_cost_per_unit['产品'] == product) &
        (allocated_cost_per_unit['周期（旬）'] == period)
    ]

    avg_allocated_transport = allocated_costs['平均分摊运输成本'].iloc[0] if not allocated_costs.empty else 0
    avg_allocated_inventory = allocated_costs['平均分摊库存成本'].iloc[0] if not allocated_costs.empty else 0

    unit_variable_cost = raw_material_cost_per_unit_val + production_cost_per_unit_val + avg_allocated_transport + avg_allocated_inventory

    unit_variable_cost_data.append({
        '工厂': factory,
        '生产线': line,
        '产品': product,
        '周期（旬）': period,
        '单位变动成本': unit_variable_cost
    })

unit_variable_cost_df = pd.DataFrame(unit_variable_cost_data)

print("\n--- 单位变动成本 (按工厂、生产线、产品、周期) ---")
if not unit_variable_cost_df.empty:
    display(unit_variable_cost_df)
else:
    print("未能计算单位变动成本。")

NameError: name 'production_cost_df_detailed' is not defined

## 计算单位边际贡献

### Subtask:
使用产品售价减去单位变动成本，计算每件产品在每条生产线上的单位边际贡献。

In [33]:
# --- 计算单位边际贡献 ---

# Merge unit variable cost with sales price data
# Sales price is per product, not per production line or period, but variable cost is.
# We'll merge based on product to get the sales price for each unit variable cost entry.

unit_contribution_margin_data = []
for index, row in unit_variable_cost_df.iterrows():
    product = row['产品']
    unit_variable_cost = row['单位变动成本']
    factory = row['工厂']
    line = row['生产线']
    period = row['周期（旬）']

    sales_price = product_data[product]['sales_price']
    unit_contribution_margin = sales_price - unit_variable_cost

    unit_contribution_margin_data.append({
        '工厂': factory,
        '生产线': line,
        '产品': product,
        '周期（旬）': period,
        '单位变动成本': unit_variable_cost,
        '销售价格': sales_price,
        '单位边际贡献': unit_contribution_margin
    })

unit_contribution_margin_df = pd.DataFrame(unit_contribution_margin_data)

print("\n--- 单位边际贡献 (按工厂、生产线、产品、周期) ---")
if not unit_contribution_margin_df.empty:
    display(unit_contribution_margin_df)
else:
    print("未能计算单位边际贡献。")


--- 单位边际贡献 (按工厂、生产线、产品、周期) ---


,工厂,生产线,产品,周期（旬）,单位变动成本,销售价格,单位边际贡献
0,北京工厂,Line_1,橙汁,1,9.796304,15.0,5.203696
1,北京工厂,Line_2,芒果汁,1,12.172972,20.0,7.827028
2,广州工厂,Line_A,橙汁,1,9.296304,15.0,5.703696
3,广州工厂,Line_A,橙汁,3,9.541935,15.0,5.458065
4,广州工厂,Line_A,芒果汁,2,11.518649,20.0,8.481351
5,广州工厂,Line_B,苹果汁,1,8.554587,13.0,4.445413
6,广州工厂,Line_B,苹果汁,3,8.840588,13.0,4.159412
7,广州工厂,Line_B,葡萄汁,2,10.648222,18.0,7.351778
8,上海工厂,Line_X,橙汁,2,10.310545,15.0,4.689455
9,上海工厂,Line_X,葡萄汁,1,11.123810,18.0,6.876190


## 计算盈亏平衡产量

### Subtask:
使用生产线激活成本除以单位边际贡献，计算每条生产线在每个周期的盈亏平衡产量，并按工厂和全部汇总。

In [34]:
# --- 计算盈亏平衡产量 ---

break_even_data = []
# Note: Break-even calculation is typically done per production line for a specific product in a specific period
# where there is a fixed cost associated with activating that line for that product in that period.
# In this model, the line_activation_cost is per line per period (旬), regardless of the product mix on the line.
# This makes a direct break-even calculation per product per line per period challenging with this fixed cost structure.

# A more meaningful break-even analysis would be based on the total fixed cost for a period (sum of activated line costs)
# and the total contribution margin generated in that period.

# Let's calculate break-even at the production line level per period, assuming the activation cost is attributed to the total production volume of that line in that period.
# This is still an approximation due to the shared fixed cost across products on a line.

# Calculate total production volume per line per period
total_production_line_period = production_plan_df.groupby(['工厂', '生产线', '周期（旬）'])['生产量'].sum().reset_index()
total_production_line_period.rename(columns={'生产量': '总生产量'}, inplace=True)


# Calculate total contribution margin per line per period
# Need to merge production plan with unit contribution margin
production_plan_with_cm = pd.merge(
    production_plan_df,
    unit_contribution_margin_df[['工厂', '生产线', '产品', '周期（旬）', '单位边际贡献']],
    on=['工厂', '生产线', '产品', '周期（旬）'],
    how='left'
)
production_plan_with_cm['总边际贡献'] = production_plan_with_cm['生产量'] * production_plan_with_cm['单位边际贡献']

total_contribution_margin_line_period = production_plan_with_cm.groupby(['工厂', '生产线', '周期（旬）'])['总边际贡献'].sum().reset_index()
total_contribution_margin_line_period.rename(columns={'总边际贡献': '总边际贡献'}, inplace=True)


# Merge total contribution margin with line activation cost
break_even_analysis_line_period = pd.merge(
    total_contribution_margin_line_period,
    total_production_line_period,
    on=['工厂', '生产线', '周期（旬）'],
    how='left'
)

# Add line activation cost
break_even_analysis_line_period['生产线激活成本'] = break_even_analysis_line_period.apply(
    lambda row: line_activation_cost.get((row['工厂'], row['生产线']), 0), axis=1
)


# Calculate Break-even Quantity per line per period
# Break-even Quantity = Fixed Cost / Unit Contribution Margin
# Using total contribution margin per line per period and assuming fixed cost is line activation cost
epsilon = 1e-9 # To avoid division by zero
break_even_analysis_line_period['盈亏平衡产量 (件)'] = break_even_analysis_line_period['生产线激活成本'] / (break_even_analysis_line_period['总边际贡献'] / (break_even_analysis_line_period['总生产量'] + epsilon) + epsilon)


print("\n--- 生产线盈亏平衡产量 (按工厂、生产线、周期) ---")
if not break_even_analysis_line_period.empty:
    display(break_even_analysis_line_period)
else:
    print("未能计算生产线盈亏平衡产量。")


# Summarize Break-even Quantity by Factory
break_even_analysis_factory = break_even_analysis_line_period.groupby('工厂')[['生产线激活成本', '总边际贡献', '总生产量']].sum().reset_index()
epsilon = 1e-9
break_even_analysis_factory['盈亏平衡产量 (件)'] = break_even_analysis_factory['生产线激活成本'] / (break_even_analysis_factory['总边际贡献'] / (break_even_analysis_factory['总生产量'] + epsilon) + epsilon)

print("\n--- 工厂盈亏平衡产量 (按工厂汇总) ---")
if not break_even_analysis_factory.empty:
    display(break_even_analysis_factory)
else:
    print("未能计算工厂盈亏平衡产量。")

# Summarize Break-even Quantity Overall
break_even_analysis_overall = break_even_analysis_factory[['生产线激活成本', '总边际贡献', '总生产量']].sum().to_frame().T
epsilon = 1e-9
break_even_analysis_overall['盈亏平衡产量 (件)'] = break_even_analysis_overall['生产线激活成本'] / (break_even_analysis_overall['总边际贡献'] / (break_even_analysis_overall['总生产量'] + epsilon) + epsilon)

print("\n--- 全部汇总盈亏平衡产量 ---")
if not break_even_analysis_overall.empty:
    display(break_even_analysis_overall)
else:
    print("未能计算全部汇总盈亏平衡产量。")


--- 生产线盈亏平衡产量 (按工厂、生产线、周期) ---


,工厂,生产线,周期（旬）,总边际贡献,总生产量,生产线激活成本,盈亏平衡产量 (件)
0,上海工厂,Line_X,1,144400.00000,21000,6000,872.576177
1,上海工厂,Line_X,2,128960.00000,27500,6000,1279.466501
2,北京工厂,Line_1,1,166518.26087,32000,5000,960.855579
3,北京工厂,Line_2,1,165933.00000,21200,4800,613.259569
4,广州工厂,Line_A,1,342221.73913,60000,4500,788.962153
5,广州工厂,Line_A,2,251048.00000,29600,4500,530.575826
6,广州工厂,Line_A,3,253800.00000,46500,4500,824.468085
7,广州工厂,Line_B,1,263613.00000,59300,4000,899.803879
8,广州工厂,Line_B,2,330830.00000,45000,4000,544.086086
9,广州工厂,Line_B,3,141420.00000,34000,4000,961.674445



--- 工厂盈亏平衡产量 (按工厂汇总) ---


,工厂,生产线激活成本,总边际贡献,总生产量,盈亏平衡产量 (件)
0,上海工厂,12000,2.733600e+05,48500,2129.060579
1,北京工厂,9800,3.324513e+05,53200,1568.229877
2,广州工厂,25500,1.582933e+06,274400,4420.402602



--- 全部汇总盈亏平衡产量 ---


,生产线激活成本,总边际贡献,总生产量,盈亏平衡产量 (件)
0,47300.0,2188744.0,376100.0,8127.734444


## 可视化盈亏平衡产量

### Subtask:
创建条形图等可视化，展示按生产线、按工厂和全部汇总的盈亏平衡产量。

In [35]:
# --- 可视化盈亏平衡产量 ---

# 可视化1: 按生产线盈亏平衡产量
if not break_even_analysis_line_period.empty:
    fig_be_line = px.bar(
        break_even_analysis_line_period,
        x='周期（旬）',
        y='盈亏平衡产量 (件)',
        color='生产线',
        facet_col='工厂',
        title='生产线盈亏平衡产量（按工厂和周期）',
        labels={'盈亏平衡产量 (件)': '盈亏平衡产量 (件)', '周期（旬）': '周期（旬)'}
    )
    fig_be_line.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_be_line.show()
else:
    print("没有生产线盈亏平衡产量数据可供可视化。")

# 可视化2: 按工厂盈亏平衡产量
if not break_even_analysis_factory.empty:
    fig_be_factory = px.bar(
        break_even_analysis_factory,
        x='工厂',
        y='盈亏平衡产量 (件)',
        title='工厂盈亏平衡产量',
        labels={'盈亏平衡产量 (件)': '盈亏平衡产量 (件)', '工厂': '工厂'}
    )
    fig_be_factory.show()
else:
    print("没有工厂盈亏平衡产量数据可供可视化。")

# 可视化3: 全部汇总盈亏平衡产量
if not break_even_analysis_overall.empty:
    fig_be_overall = px.bar(
        break_even_analysis_overall,
        x=break_even_analysis_overall.index, # Use index as x-axis for single row
        y='盈亏平衡产量 (件)',
        title='全部汇总盈亏平衡产量',
        labels={'盈亏平衡产量 (件)': '盈亏平衡产量 (件)'}
    )
    fig_be_overall.update_xaxes(visible=False) # Hide x-axis as it's just an index
    fig_be_overall.show()
else:
    print("没有全部汇总盈亏平衡产量数据可供可视化。")

# Task
计算并可视化每条生产线在每个周期内，仅考虑生产成本和激活成本的情况下，达到盈亏平衡所需的产量。

## 计算单位生产成本

### Subtask:
提取每件产品在每条生产线上的单位生产成本。


**Reasoning**:
Extract the unit production cost for each product on each production line according to the instructions.



In [36]:
# --- 提取单位生产成本 ---

production_cost_per_unit_data = []
for f in factories:
    for line in production_lines_info[f]:
        for p in production_lines_info[f][line]['products']:
            cost = production_cost.get((f, line, p), 0)
            production_cost_per_unit_data.append({
                '工厂': f,
                '生产线': line,
                '产品': p,
                '单位生产成本': cost
            })

production_cost_per_unit_df = pd.DataFrame(production_cost_per_unit_data)

print("\n--- 单位生产成本 (按工厂、生产线、产品) ---")
if not production_cost_per_unit_df.empty:
    display(production_cost_per_unit_df)
else:
    print("未能提取单位生产成本数据。")


--- 单位生产成本 (按工厂、生产线、产品) ---


,工厂,生产线,产品,单位生产成本
0,北京工厂,Line_1,橙汁,8.0
1,北京工厂,Line_1,葡萄汁,9.0
2,北京工厂,Line_2,苹果汁,7.8
3,北京工厂,Line_2,芒果汁,9.5
4,广州工厂,Line_A,橙汁,7.5
5,广州工厂,Line_A,芒果汁,9.2
6,广州工厂,Line_B,苹果汁,7.0
7,广州工厂,Line_B,葡萄汁,8.5
8,上海工厂,Line_X,橙汁,8.2
9,上海工厂,Line_X,苹果汁,7.5


**Reasoning**:
The subtask of extracting unit production costs is complete. The next step is to calculate the unit variable cost, which was outlined as combining raw material cost, production cost, and allocated transportation and inventory costs.



In [37]:
# --- 计算单位变动成本 ---

# This is a complex step as costs need to be attributed back to the production line level per unit.
# We will make a simplifying assumption:
# Variable Cost per Unit = Raw Material Cost + Production Cost per Unit + Allocated Transportation Cost per Unit + Allocated Inventory Cost per Unit

# Allocate transportation and inventory costs back to the production output.
# This requires knowing which production from which line/factory ends up where.
# A simplified approach is to distribute total transport/inventory cost for a product in a period
# based on the proportion of production volume from each line for that product in that period.

# Calculate total production volume for each product and period
total_production_prod_period = production_cost_df_detailed.groupby(['产品', '周期（旬）'])['生产量'].sum().reset_index()
total_production_prod_period.rename(columns={'生产量': '总生产量'}, inplace=True)

# Calculate allocated transport and inventory cost per unit for each product and period
# Total transport cost per product per period
total_transport_cost_prod_period = transport_cost_df_detailed.groupby(['产品', '周期（旬）'])['成本'].sum().reset_index()
total_transport_cost_prod_period.rename(columns={'成本': '总运输成本'}, inplace=True)

# Total inventory cost per product per period
total_inventory_cost_prod_period = inventory_cost_df_detailed.groupby(['产品', '周期（旬）'])['成本'].sum().reset_index()
total_inventory_cost_prod_period.rename(columns={'成本': '总库存成本'}, inplace=True)


# Merge total costs with total production to get average allocated cost per unit
allocated_cost_per_unit = pd.merge(total_production_prod_period, total_transport_cost_prod_period, on=['产品', '周期（旬）'], how='left')
allocated_cost_per_unit = pd.merge(allocated_cost_per_unit, total_inventory_cost_prod_period, on=['产品', '周期（旬）'], how='left')
allocated_cost_per_unit.fillna(0, inplace=True)

# Calculate average allocated cost per unit (transport + inventory)
# Add a small epsilon to total_生产量 to avoid division by zero
epsilon = 1e-9
allocated_cost_per_unit['平均分摊运输成本'] = allocated_cost_per_unit['总运输成本'] / (allocated_cost_per_unit['总生产量'] + epsilon)
allocated_cost_per_unit['平均分摊库存成本'] = allocated_cost_per_unit['总库存成本'] / (allocated_cost_per_unit['总生产量'] + epsilon)

# Now calculate unit variable cost for each production entry
unit_variable_cost_data = []
for index, row in production_cost_df_detailed.iterrows():
    product = row['产品']
    period = row['周期（旬）']
    factory = row['工厂']
    line = row['生产线']
    production_cost_per_unit_val = production_cost.get((factory, line, product), 0)
    raw_material_cost_per_unit_val = product_data[product]['raw_material_cost']

    # Find the corresponding allocated costs
    allocated_costs = allocated_cost_per_unit[
        (allocated_cost_per_unit['产品'] == product) &
        (allocated_cost_per_unit['周期（旬）'] == period)
    ]

    avg_allocated_transport = allocated_costs['平均分摊运输成本'].iloc[0] if not allocated_costs.empty else 0
    avg_allocated_inventory = allocated_costs['平均分摊库存成本'].iloc[0] if not allocated_costs.empty else 0

    unit_variable_cost = raw_material_cost_per_unit_val + production_cost_per_unit_val + avg_allocated_transport + avg_allocated_inventory

    unit_variable_cost_data.append({
        '工厂': factory,
        '生产线': line,
        '产品': product,
        '周期（旬）': period,
        '单位变动成本': unit_variable_cost
    })

unit_variable_cost_df = pd.DataFrame(unit_variable_cost_data)

print("\n--- 单位变动成本 (按工厂、生产线、产品、周期) ---")
if not unit_variable_cost_df.empty:
    display(unit_variable_cost_df)
else:
    print("未能计算单位变动成本。")


--- 单位变动成本 (按工厂、生产线、产品、周期) ---


,工厂,生产线,产品,周期（旬）,单位变动成本
0,北京工厂,Line_1,橙汁,1,9.796304
1,北京工厂,Line_2,芒果汁,1,12.172972
2,广州工厂,Line_A,橙汁,1,9.296304
3,广州工厂,Line_A,橙汁,3,9.541935
4,广州工厂,Line_A,芒果汁,2,11.518649
5,广州工厂,Line_B,苹果汁,1,8.554587
6,广州工厂,Line_B,苹果汁,3,8.840588
7,广州工厂,Line_B,葡萄汁,2,10.648222
8,上海工厂,Line_X,橙汁,2,10.310545
9,上海工厂,Line_X,葡萄汁,1,11.123810


**Reasoning**:
Calculate the unit contribution margin by subtracting the unit variable cost from the sales price.



In [38]:
# --- 计算单位边际贡献 ---

# Merge unit variable cost with sales price data
# Sales price is per product, not per production line or period, but variable cost is.
# We'll merge based on product to get the sales price for each unit variable cost entry.

unit_contribution_margin_data = []
for index, row in unit_variable_cost_df.iterrows():
    product = row['产品']
    unit_variable_cost = row['单位变动成本']
    factory = row['工厂']
    line = row['生产线']
    period = row['周期（旬）']

    sales_price = product_data[product]['sales_price']
    unit_contribution_margin = sales_price - unit_variable_cost

    unit_contribution_margin_data.append({
        '工厂': factory,
        '生产线': line,
        '产品': product,
        '周期（旬）': period,
        '单位变动成本': unit_variable_cost,
        '销售价格': sales_price,
        '单位边际贡献': unit_contribution_margin
    })

unit_contribution_margin_df = pd.DataFrame(unit_contribution_margin_data)

print("\n--- 单位边际贡献 (按工厂、生产线、产品、周期) ---")
if not unit_contribution_margin_df.empty:
    display(unit_contribution_margin_df)
else:
    print("未能计算单位边际贡献。")


--- 单位边际贡献 (按工厂、生产线、产品、周期) ---


,工厂,生产线,产品,周期（旬）,单位变动成本,销售价格,单位边际贡献
0,北京工厂,Line_1,橙汁,1,9.796304,15.0,5.203696
1,北京工厂,Line_2,芒果汁,1,12.172972,20.0,7.827028
2,广州工厂,Line_A,橙汁,1,9.296304,15.0,5.703696
3,广州工厂,Line_A,橙汁,3,9.541935,15.0,5.458065
4,广州工厂,Line_A,芒果汁,2,11.518649,20.0,8.481351
5,广州工厂,Line_B,苹果汁,1,8.554587,13.0,4.445413
6,广州工厂,Line_B,苹果汁,3,8.840588,13.0,4.159412
7,广州工厂,Line_B,葡萄汁,2,10.648222,18.0,7.351778
8,上海工厂,Line_X,橙汁,2,10.310545,15.0,4.689455
9,上海工厂,Line_X,葡萄汁,1,11.123810,18.0,6.876190


**Reasoning**:
Calculate the break-even quantity for each production line per period, and then summarize this by factory and overall, as requested in the original task.



In [39]:
# --- 计算盈亏平衡产量 ---

break_even_data = []
# Note: Break-even calculation is typically done per production line for a specific product in a specific period
# where there is a fixed cost associated with activating that line for that product in that period.
# In this model, the line_activation_cost is per line per period (旬), regardless of the product mix on the line.
# This makes a direct break-even calculation per product per line per period challenging with this fixed cost structure.

# A more meaningful break-even analysis would be based on the total fixed cost for a period (sum of activated line costs)
# and the total contribution margin generated in that period.

# Let's calculate break-even at the production line level per period, assuming the activation cost is attributed to the total production volume of that line in that period.
# This is still an approximation due to the shared fixed cost across products on a line.

# Calculate total production volume per line per period
total_production_line_period = production_plan_df.groupby(['工厂', '生产线', '周期（旬）'])['生产量'].sum().reset_index()
total_production_line_period.rename(columns={'生产量': '总生产量'}, inplace=True)


# Calculate total contribution margin per line per period
# Need to merge production plan with unit contribution margin
production_plan_with_cm = pd.merge(
    production_plan_df,
    unit_contribution_margin_df[['工厂', '生产线', '产品', '周期（旬）', '单位边际贡献']],
    on=['工厂', '生产线', '产品', '周期（旬）'],
    how='left'
)
production_plan_with_cm['总边际贡献'] = production_plan_with_cm['生产量'] * production_plan_with_cm['单位边际贡献']

total_contribution_margin_line_period = production_plan_with_cm.groupby(['工厂', '生产线', '周期（旬）'])['总边际贡献'].sum().reset_index()
total_contribution_margin_line_period.rename(columns={'总边际贡献': '总边际贡献'}, inplace=True)


# Merge total contribution margin with line activation cost
break_even_analysis_line_period = pd.merge(
    total_contribution_margin_line_period,
    total_production_line_period,
    on=['工厂', '生产线', '周期（旬）'],
    how='left'
)

# Add line activation cost
break_even_analysis_line_period['生产线激活成本'] = break_even_analysis_line_period.apply(
    lambda row: line_activation_cost.get((row['工厂'], row['生产线']), 0), axis=1
)


# Calculate Break-even Quantity per line per period
# Break-even Quantity = Fixed Cost / Unit Contribution Margin
# Using total contribution margin per line per period and assuming fixed cost is line activation cost
epsilon = 1e-9 # To avoid division by zero
break_even_analysis_line_period['盈亏平衡产量 (件)'] = break_even_analysis_line_period['生产线激活成本'] / (break_even_analysis_line_period['总边际贡献'] / (break_even_analysis_line_period['总生产量'] + epsilon) + epsilon)


print("\n--- 生产线盈亏平衡产量 (按工厂、生产线、周期) ---")
if not break_even_analysis_line_period.empty:
    display(break_even_analysis_line_period)
else:
    print("未能计算生产线盈亏平衡产量。")


# Summarize Break-even Quantity by Factory
break_even_analysis_factory = break_even_analysis_line_period.groupby('工厂')[['生产线激活成本', '总边际贡献', '总生产量']].sum().reset_index()
epsilon = 1e-9
break_even_analysis_factory['盈亏平衡产量 (件)'] = break_even_analysis_factory['生产线激活成本'] / (break_even_analysis_factory['总边际贡献'] / (break_even_analysis_factory['总生产量'] + epsilon) + epsilon)

print("\n--- 工厂盈亏平衡产量 (按工厂汇总) ---")
if not break_even_analysis_factory.empty:
    display(break_even_analysis_factory)
else:
    print("未能计算工厂盈亏平衡产量。")

# Summarize Break-even Quantity Overall
break_even_analysis_overall = break_even_analysis_factory[['生产线激活成本', '总边际贡献', '总生产量']].sum().to_frame().T
epsilon = 1e-9
break_even_analysis_overall['盈亏平衡产量 (件)'] = break_even_analysis_overall['生产线激活成本'] / (break_even_analysis_overall['总边际贡献'] / (break_even_analysis_overall['总生产量'] + epsilon) + epsilon)

print("\n--- 全部汇总盈亏平衡产量 ---")
if not break_even_analysis_overall.empty:
    display(break_even_analysis_overall)
else:
    print("未能计算全部汇总盈亏平衡产量。")


--- 生产线盈亏平衡产量 (按工厂、生产线、周期) ---


,工厂,生产线,周期（旬）,总边际贡献,总生产量,生产线激活成本,盈亏平衡产量 (件)
0,上海工厂,Line_X,1,144400.00000,21000,6000,872.576177
1,上海工厂,Line_X,2,128960.00000,27500,6000,1279.466501
2,北京工厂,Line_1,1,166518.26087,32000,5000,960.855579
3,北京工厂,Line_2,1,165933.00000,21200,4800,613.259569
4,广州工厂,Line_A,1,342221.73913,60000,4500,788.962153
5,广州工厂,Line_A,2,251048.00000,29600,4500,530.575826
6,广州工厂,Line_A,3,253800.00000,46500,4500,824.468085
7,广州工厂,Line_B,1,263613.00000,59300,4000,899.803879
8,广州工厂,Line_B,2,330830.00000,45000,4000,544.086086
9,广州工厂,Line_B,3,141420.00000,34000,4000,961.674445



--- 工厂盈亏平衡产量 (按工厂汇总) ---


,工厂,生产线激活成本,总边际贡献,总生产量,盈亏平衡产量 (件)
0,上海工厂,12000,2.733600e+05,48500,2129.060579
1,北京工厂,9800,3.324513e+05,53200,1568.229877
2,广州工厂,25500,1.582933e+06,274400,4420.402602



--- 全部汇总盈亏平衡产量 ---


,生产线激活成本,总边际贡献,总生产量,盈亏平衡产量 (件)
0,47300.0,2188744.0,376100.0,8127.734444


**Reasoning**:
Visualize the calculated break-even quantities at the production line, factory, and overall levels using bar charts. Also, include the visualizations for production line and warehouse utilization rates as requested in the overall task description.



In [40]:
# --- 可视化盈亏平衡产量 ---

# 可视化1: 按生产线盈亏平衡产量
if not break_even_analysis_line_period.empty:
    fig_be_line = px.bar(
        break_even_analysis_line_period,
        x='周期（旬）',
        y='盈亏平衡产量 (件)',
        color='生产线',
        facet_col='工厂',
        title='生产线盈亏平衡产量（按工厂和周期）',
        labels={'盈亏平衡产量 (件)': '盈亏平衡产量 (件)', '周期（旬）': '周期（旬)'}
    )
    fig_be_line.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_be_line.show()
else:
    print("没有生产线盈亏平衡产量数据可供可视化。")

# 可视化2: 按工厂盈亏平衡产量
if not break_even_analysis_factory.empty:
    fig_be_factory = px.bar(
        break_even_analysis_factory,
        x='工厂',
        y='盈亏平衡产量 (件)',
        title='工厂盈亏平衡产量',
        labels={'盈亏平衡产量 (件)': '盈亏平衡产量 (件)', '工厂': '工厂'}
    )
    fig_be_factory.show()
else:
    print("没有工厂盈亏平衡产量数据可供可视化。")

# 可视化3: 全部汇总盈亏平衡产量
if not break_even_analysis_overall.empty:
    fig_be_overall = px.bar(
        break_even_analysis_overall,
        x=break_even_analysis_overall.index, # Use index as x-axis for single row
        y='盈亏平衡产量 (件)',
        title='全部汇总盈亏平衡产量',
        labels={'盈亏平衡产量 (件)': '盈亏平衡产量 (件)'}
    )
    fig_be_overall.update_xaxes(visible=False) # Hide x-axis as it's just an index
    fig_be_overall.show()
else:
    print("没有全部汇总盈亏平衡产量数据可供可视化。")

# --- 可视化资源利用率 (from previous analysis) ---

# 可视化生产线利用率
if not line_utilization_df.empty:
    fig_line_utilization = px.bar(
        line_utilization_df,
        x='周期（旬）',
        y='利用率',
        color='产品',
        facet_col='工厂',
        facet_row='生产线',
        title='生产线利用率（按工厂、生产线、产品和周期）',
        labels={'利用率': '利用率', '周期（旬）': '周期（旬）'}
    )
    fig_line_utilization.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_line_utilization.show()
else:
    print("没有可展示的生产线利用率数据。")


# 可视化仓库空间利用率
if not warehouse_utilization_df.empty:
    fig_warehouse_utilization = px.bar(
        warehouse_utilization_df,
        x='周期（旬）',
        y='利用率',
        color='分仓',
        barmode='group',
        title='仓库空间利用率（按分仓和周期）',
        labels={'利用率': '利用率', '周期（旬）': '周期（旬）'}
    )
    fig_warehouse_utilization.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_warehouse_utilization.show()
else:
    print("没有可展示的仓库空间利用率数据。")

## 计算单位边际贡献（仅考虑生产成本）

### Subtask:
使用产品售价减去单位生产成本，计算单位边际贡献。

In [41]:
# --- 计算单位边际贡献（仅考虑生产成本） ---

# Merge unit production cost with sales price data
unit_contribution_margin_prod_cost_data = []
for index, row in production_cost_per_unit_df.iterrows():
    product = row['产品']
    factory = row['工厂']
    line = row['生产线']
    unit_production_cost = row['单位生产成本']

    sales_price = product_data[product]['sales_price']
    unit_contribution_margin_prod_cost = sales_price - unit_production_cost

    unit_contribution_margin_prod_cost_data.append({
        '工厂': factory,
        '生产线': line,
        '产品': product,
        '单位生产成本': unit_production_cost,
        '销售价格': sales_price,
        '单位边际贡献 (仅生产成本)': unit_contribution_margin_prod_cost
    })

unit_contribution_margin_prod_cost_df = pd.DataFrame(unit_contribution_margin_prod_cost_data)

print("\n--- 单位边际贡献 (仅考虑生产成本，按工厂、生产线、产品) ---")
if not unit_contribution_margin_prod_cost_df.empty:
    display(unit_contribution_margin_prod_cost_df)
else:
    print("未能计算单位边际贡献（仅考虑生产成本）。")


--- 单位边际贡献 (仅考虑生产成本，按工厂、生产线、产品) ---


,工厂,生产线,产品,单位生产成本,销售价格,单位边际贡献 (仅生产成本)
0,北京工厂,Line_1,橙汁,8.0,15.0,7.0
1,北京工厂,Line_1,葡萄汁,9.0,18.0,9.0
2,北京工厂,Line_2,苹果汁,7.8,13.0,5.2
3,北京工厂,Line_2,芒果汁,9.5,20.0,10.5
4,广州工厂,Line_A,橙汁,7.5,15.0,7.5
5,广州工厂,Line_A,芒果汁,9.2,20.0,10.8
6,广州工厂,Line_B,苹果汁,7.0,13.0,6.0
7,广州工厂,Line_B,葡萄汁,8.5,18.0,9.5
8,上海工厂,Line_X,橙汁,8.2,15.0,6.8
9,上海工厂,Line_X,苹果汁,7.5,13.0,5.5


## 计算盈亏平衡产量（仅考虑生产成本和激活成本）

### Subtask:
使用生产线激活成本除以单位边际贡献（仅考虑生产成本），计算每条生产线在每个周期的盈亏平衡产量。

In [42]:
# --- 计算盈亏平衡产量（仅考虑生产成本和激活成本） ---

# We need to calculate the break-even quantity per production line per period,
# considering only the line activation cost (fixed) and the contribution margin based on production cost (variable).

break_even_prod_act_cost_data = []

# Iterate through each production line and period where production occurred
# We need to find the activated lines in each period
activated_lines_per_period = defaultdict(set)
for f in factories:
    for line in production_lines_info[f]:
        for t in periods:
            # Check if the line was "on" in this period based on the model results
            # We assume a line is "on" if any product was produced on it in that period
            total_produced_on_line_in_period = sum(
                 value(produce[f][line][p][t]) if value(produce[f][line][p][t]) is not None else 0
                 for p in production_lines_info[f][line]['products']
            )
            if total_produced_on_line_in_period > 0:
                 activated_lines_per_period[(f, line, t)].add(line)


for (factory, line, period) in activated_lines_per_period:
    # Get the activation cost for this line and period
    activation_cost = line_activation_cost.get((factory, line), 0)

    # We need the average unit contribution margin (only production cost) for the products produced on this line in this period.
    # This is an approximation as different products on the same line have different contribution margins.
    # A more precise method would involve the exact mix of products produced.
    # For this simplified calculation, we'll average the unit contribution margin for all products that *can* be produced on this line.

    # Filter the unit_contribution_margin_prod_cost_df for products produced on this line
    prods_on_this_line = production_lines_info[factory][line]['products']
    cm_for_this_line_prods = unit_contribution_margin_prod_cost_df[
        (unit_contribution_margin_prod_cost_df['工厂'] == factory) &
        (unit_contribution_margin_prod_cost_df['生产线'] == line) &
        (unit_contribution_margin_prod_cost_df['产品'].isin(prods_on_this_line))
    ]

    average_unit_cm = cm_for_this_line_prods['单位边际贡献 (仅生产成本)'].mean() if not cm_for_this_line_prods.empty else 0

    # Calculate break-even quantity
    break_even_quantity = activation_cost / (average_unit_cm + epsilon) if average_unit_cm > 0 else float('inf') # Add epsilon for safety

    break_even_prod_act_cost_data.append({
        '工厂': factory,
        '生产线': line,
        '周期（旬）': period,
        '生产线激活成本': activation_cost,
        '平均单位边际贡献 (仅生产成本)': average_unit_cm,
        '盈亏平衡产量 (件, 仅生产+激活成本)': break_even_quantity
    })

break_even_prod_act_cost_df = pd.DataFrame(break_even_prod_act_cost_data)

print("\n--- 生产线盈亏平衡产量 (仅考虑生产+激活成本, 按工厂、生产线、周期) ---")
if not break_even_prod_act_cost_df.empty:
    display(break_even_prod_act_cost_df)
else:
    print("未能计算生产线盈亏平衡产量（仅考虑生产+激活成本）。")

# Summarize by Factory (sum of activation costs, weighted average CM)
break_even_factory_prod_act_cost_data = []
for f in factories:
    factory_lines = [line for line in production_lines_info[f]]
    total_activation_cost_factory = sum(line_activation_cost.get((f, line), 0) for line in factory_lines)

    # Calculate weighted average unit CM for the factory based on total production volume
    factory_production_for_cm = production_plan_with_cm[production_plan_with_cm['工厂'] == f]
    if not factory_production_for_cm.empty:
        total_cm_factory = factory_production_for_cm['总边际贡献'].sum()
        total_prod_factory = factory_production_for_cm['生产量'].sum()
        weighted_avg_cm_factory = total_cm_factory / (total_prod_factory + epsilon)
    else:
        weighted_avg_cm_factory = 0


    # Calculate break-even quantity for the factory (approximation)
    break_even_quantity_factory = total_activation_cost_factory / (weighted_avg_cm_factory + epsilon) if weighted_avg_cm_factory > 0 else float('inf')

    break_even_factory_prod_act_cost_data.append({
        '工厂': f,
        '总生产线激活成本': total_activation_cost_factory,
        '加权平均单位边际贡献': weighted_avg_cm_factory,
        '工厂盈亏平衡产量 (件, 仅生产+激活成本)': break_even_quantity_factory
    })

break_even_factory_prod_act_cost_df = pd.DataFrame(break_even_factory_prod_act_cost_data)

print("\n--- 工厂盈亏平衡产量 (仅考虑生产+激活成本, 按工厂汇总) ---")
if not break_even_factory_prod_act_cost_df.empty:
    display(break_even_factory_prod_act_cost_df)
else:
    print("未能计算工厂盈亏平衡产量（仅考虑生产+激活成本）。")


# Summarize Overall (sum of all activation costs, weighted average CM)
total_overall_activation_cost = sum(line_activation_cost.values())

# Calculate weighted average unit CM for overall based on total production volume
if not production_plan_with_cm.empty:
    total_cm_overall = production_plan_with_cm['总边际贡献'].sum()
    total_prod_overall = production_plan_with_cm['生产量'].sum()
    weighted_avg_cm_overall = total_cm_overall / (total_prod_overall + epsilon)
else:
    weighted_avg_cm_overall = 0

# Calculate break-even quantity overall (approximation)
break_even_quantity_overall = total_overall_activation_cost / (weighted_avg_cm_overall + epsilon) if weighted_avg_cm_overall > 0 else float('inf')

break_even_overall_prod_act_cost_data = [{
    '总生产线激活成本': total_overall_activation_cost,
    '加权平均单位边际贡献': weighted_avg_cm_overall,
    '全部汇总盈亏平衡产量 (件, 仅生产+激活成本)': break_even_quantity_overall
}]

break_even_overall_prod_act_cost_df = pd.DataFrame(break_even_overall_prod_act_cost_data)

print("\n--- 全部汇总盈亏平衡产量 (仅考虑生产+激活成本) ---")
if not break_even_overall_prod_act_cost_df.empty:
    display(break_even_overall_prod_act_cost_df)
else:
    print("未能计算全部汇总盈亏平衡产量（仅考虑生产+激活成本）。")


--- 生产线盈亏平衡产量 (仅考虑生产+激活成本, 按工厂、生产线、周期) ---


,工厂,生产线,周期（旬）,生产线激活成本,平均单位边际贡献 (仅生产成本),"盈亏平衡产量 (件, 仅生产+激活成本)"
0,北京工厂,Line_1,1,5000,8.000,625.000000
1,北京工厂,Line_2,1,4800,7.850,611.464968
2,广州工厂,Line_A,1,4500,9.150,491.803279
3,广州工厂,Line_A,2,4500,9.150,491.803279
4,广州工厂,Line_A,3,4500,9.150,491.803279
5,广州工厂,Line_B,1,4000,7.750,516.129032
6,广州工厂,Line_B,2,4000,7.750,516.129032
7,广州工厂,Line_B,3,4000,7.750,516.129032
8,上海工厂,Line_X,1,6000,7.925,757.097792
9,上海工厂,Line_X,2,6000,7.925,757.097792



--- 工厂盈亏平衡产量 (仅考虑生产+激活成本, 按工厂汇总) ---


,工厂,总生产线激活成本,加权平均单位边际贡献,"工厂盈亏平衡产量 (件, 仅生产+激活成本)"
0,北京工厂,9800,6.249084,1568.229877
1,广州工厂,8500,5.768705,1473.467534
2,上海工厂,6000,5.636289,1064.530290



--- 全部汇总盈亏平衡产量 (仅考虑生产+激活成本) ---


,总生产线激活成本,加权平均单位边际贡献,"全部汇总盈亏平衡产量 (件, 仅生产+激活成本)"
0,24300,5.81958,4175.559133


## 可视化盈亏平衡产量

### Subtask:
创建可视化展示每条生产线在每个周期的盈亏平衡产量。

In [23]:
# --- 可视化盈亏平衡产量 (仅考虑生产成本和激活成本) ---

if not break_even_prod_act_cost_df.empty:
    fig_be_prod_act_line = px.bar(
        break_even_prod_act_cost_df,
        x='周期（旬）',
        y='盈亏平衡产量 (件, 仅生产+激活成本)',
        color='生产线',
        facet_col='工厂',
        title='生产线盈亏平衡产量（仅考虑生产+激活成本，按工厂和周期）',
        labels={'盈亏平衡产量 (件, 仅生产+激活成本)': '盈亏平衡产量 (件)', '周期（旬）': '周期（旬)'}
    )
    fig_be_prod_act_line.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_be_prod_act_line.show()
else:
    print("没有盈亏平衡产量（仅考虑生产+激活成本）数据可供可视化。")

# 可视化按工厂汇总的盈亏平衡产量 (仅考虑生产+激活成本)
# Add '周期（旬）' as color for factory summary if it exists in the summarized data
# Note: The current factory summary is aggregated across periods, so adding '周期（旬）' as a dimension might not be meaningful here.
# We will keep the factory summary as is, summarizing across periods.
if not break_even_factory_prod_act_cost_df.empty:
    fig_be_prod_act_factory = px.bar(
        break_even_factory_prod_act_cost_df,
        x='工厂',
        y='工厂盈亏平衡产量 (件, 仅生产+激活成本)',
        title='工厂盈亏平衡产量（仅考虑生产+激活成本）',
        labels={'工厂盈亏平衡产量 (件, 仅生产+激活成本)': '盈亏平衡产量 (件)', '工厂': '工厂'}
    )
    fig_be_prod_act_factory.show()
else:
    print("没有工厂盈亏平衡产量（仅考虑生产+激活成本）数据可供可视化。")

# 可视化全部汇总的盈亏平衡产量 (仅考虑生产+激活成本)
# The overall summary is also aggregated across periods, so adding '周期（旬）' as a dimension might not be meaningful here.
# We will keep the overall summary as is.
if not break_even_overall_prod_act_cost_df.empty:
    fig_be_prod_act_overall = px.bar(
        break_even_overall_prod_act_cost_df,
        x=break_even_overall_prod_act_cost_df.index, # Use index as x-axis for single row
        y='全部汇总盈亏平衡产量 (件, 仅生产+激活成本)',
        title='全部汇总盈亏平衡产量（仅考虑生产+激活成本）',
        labels={'全部汇总盈亏平衡产量 (件, 仅生产+激活成本)': '盈亏平衡产量 (件)'}
    )
    fig_be_prod_act_overall.update_xaxes(visible=False) # Hide x-axis as it's just an index
    fig_be_prod_act_overall.show()
else:
    print("没有全部汇总盈亏平衡产量（仅考虑生产+激活成本）数据可供可视化。")

NameError: name 'break_even_prod_act_cost_df' is not defined

In [2]:
# Re-run Data Preparation cell to ensure variables are defined
# --- 1. 数据准备 ---

# 定义所有站点、产品和周期
factories = ['北京工厂', '广州工厂', '上海工厂'] # 工厂列表
warehouses = ['上海分仓', '武汉分仓', '成都分仓'] # 分仓列表
customers = ['沈阳客户', '深圳客户', '成都客户', '西安客户', '杭州客户'] # 客户列表
periods = [1, 2, 3]  # 三旬，每个周期代表一旬 (约10天)
days_per_period = 10 # 每旬10天
products = ['橙汁', '苹果汁', '葡萄汁', '芒果汁'] # 产品列表

# 产品数据（售价、重量、体积
product_data = {
    # '产品名称': {'sales_price': 销售价格 (元/件), 'weight': 重量 (kg/件), 'volume': 体积 (m^3/件)}
    '橙汁': {'sales_price': 8.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '苹果汁': {'sales_price': 7.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '葡萄汁': {'sales_price': 9.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '芒果汁': {'sales_price': 10.0, 'weight': 0.5, 'volume': 0.0005} # 假设每件500ml
}

# 生产线信息
production_lines_info = {
    '北京工厂': {
        'Line_1': {
            'products': ['橙汁', '葡萄汁'], # 该生产线可以生产的产品
            'min_daily_output': {'橙汁': 2000, '葡萄汁': 1800}, # 每日最小产值 (件/天)
            'max_daily_output': {'橙汁': 5000, '葡萄汁': 4500} # 每日最大产值 (件/天)
        },
        'Line_2': {
            'products': ['苹果汁', '芒果汁'],
            'min_daily_output': {'苹果汁': 1500, '芒果汁': 1200},
            'max_daily_output': {'苹果汁': 4000, '芒果汁': 3000}
        }
    },
    '广州工厂': {
        'Line_A': {
            'products': ['橙汁', '芒果汁'],
            'min_daily_output': {'橙汁': 1500, '芒果汁': 1000},
            'max_daily_output': {'橙汁': 6000, '芒果汁': 5000}
        },
        'Line_B': {
            'products': ['苹果汁', '葡萄汁'],
            'min_daily_output': {'苹果汁': 1800, '葡萄汁': 1600},
            'max_daily_output': {'苹果汁': 6500, '葡萄汁': 5500}
        }
    },
    '上海工厂': {
        'Line_X': {
            'products': ['橙汁', '苹果汁', '葡萄汁', '芒果汁'],
            'min_daily_output': {'橙汁': 1000, '苹果汁': 800, '葡萄汁': 900, '芒果汁': 700},
            'max_daily_output': {'橙汁': 4000, '苹果汁': 3500, '葡萄汁': 3800, '芒果汁': 3000}
        }
    }
}

all_lines = list(set([line for f in factories for line in production_lines_info[f]])) # 所有生产线的集合


# 成本数据 (单位: 元/件)
# 生产成本: 每件产品在特定工厂的特定生产线上生产的成本 (元/件)
# 成本包含原料、人工及其他变动生产成本。
production_cost = {
    ('北京工厂', 'Line_1', '橙汁'): 1.5 + 1.0 + 2.0, # 原料 + 人工 + 其他变动成本
    ('北京工厂', 'Line_1', '葡萄汁'): 1.8 + 1.1 + 2.2,
    ('北京工厂', 'Line_2', '苹果汁'): 1.2 + 0.9 + 1.8,
    ('北京工厂', 'Line_2', '芒果汁'): 2.0 + 1.2 + 2.5,
    ('广州工厂', 'Line_A', '橙汁'): 1.5 + 1.0 + 1.9,
    ('广州工厂', 'Line_A', '芒果汁'): 2.0 + 1.2 + 2.4,
    ('广州工厂', 'Line_B', '苹果汁'): 1.2 + 0.9 + 1.7,
    ('广州工厂', 'Line_B', '葡萄汁'): 1.8 + 1.1 + 2.1,
    ('上海工厂', 'Line_X', '橙汁'): 1.5 + 1.0 + 2.1,
    ('上海工厂', 'Line_X', '苹果汁'): 1.2 + 0.9 + 1.9,
    ('上海工厂', 'Line_X', '葡萄汁'): 1.8 + 1.1 + 2.3,
    ('上海工厂', 'Line_X', '芒果汁'): 2.0 + 1.2 + 2.6
}

# 生产线激活成本 (单位: 元/旬)
# 每旬激活一条生产线所需的固定成本 (元/旬)
line_activation_cost = {
    ('北京工厂', 'Line_1'): 5000, ('北京工厂', 'Line_2'): 4800,
    ('广州工厂', 'Line_A'): 4500, ('广州工厂', 'Line_B'): 4000,
    ('上海工厂', 'Line_X'): 6000
}


# 仓储成本 (单位: 元/件/旬)
# 每件产品在工厂仓库中存放一旬的成本 (元/件/旬)
factory_inventory_cost = {
    '北京工厂': {'橙汁': 0.08, '苹果汁': 0.10, '葡萄汁': 0.09, '芒果汁': 0.11},
    '广州工厂': {'橙汁': 0.07, '苹果汁': 0.09, '葡萄汁': 0.08, '芒果汁': 0.10},
    '上海工厂': {'橙汁': 0.09, '苹果汁': 0.11, '葡萄汁': 0.10, '芒果汁': 0.12}
}

# 每件产品在分仓中存放一旬的成本 (元/件/旬)
warehouse_inventory_cost = {
    '上海分仓': {'橙汁': 0.1, '苹果汁': 0.12, '葡萄汁': 0.11, '芒果汁': 0.13},
    '武汉分仓': {'橙汁': 0.09, '苹果汁': 0.11, '葡萄汁': 0.10, '芒果汁': 0.12},
    '成都分仓': {'橙汁': 0.11, '苹果汁': 0.13, '葡萄汁': 0.12, '芒果汁': 0.14}
}

# 运输成本 (单位: 元/件)
# 工厂到分仓的运输成本 (元/件)
transport_cost_fw = {
    ('北京工厂', '上海分仓'): 0.5, ('北京工厂', '武汉分仓'): 0.6, ('北京工厂', '成都分仓'): 0.9,
    ('广州工厂', '上海分仓'): 0.4, ('广州工厂', '武汉分仓'): 0.3, ('广州工厂', '成都分仓'): 0.7,
    ('上海工厂', '上海分仓'): 0.1, ('上海工厂', '武汉分仓'): 0.5, ('上海工厂', '成都分仓'): 0.8
}
# 工厂到客户的运输成本 (元/件)
transport_cost_fc = {
    ('北京工厂', '沈阳客户'): 0.2, ('广州工厂', '沈阳客户'): 1.5, ('上海工厂', '沈阳客户'): 0.8,
    ('北京工厂', '深圳客户'): 1.8, ('广州工厂', '深圳客户'): 0.2, ('上海工厂', '深圳客户'): 0.7,
    ('北京工厂', '成都客户'): 1.2, ('广州工厂', '成都客户'): 0.9, ('上海工厂', '成都客户'): 0.4,
    ('北京工厂', '西安客户'): 0.7, ('广州工厂', '西安客户'): 1.1, ('上海工厂', '西安客户'): 0.6,
    ('北京工厂', '杭州客户'): 0.9, ('广州工厂', '杭州客户'): 0.8, ('上海工厂', '杭州客户'): 0.3
}
# 分仓到客户的运输成本 (元/件)
transport_cost_wc = {
    ('上海分仓', '沈阳客户'): 0.8, ('武汉分仓', '沈阳客户'): 1.3, ('成都分仓', '沈阳客户'): 1.5,
    ('上海分仓', '深圳客户'): 0.6, ('武汉分仓', '深圳客户'): 0.7, ('成都分仓', '成都客户'): 1.0,
    ('上海分仓', '成都客户'): 0.5, ('武汉分仓', '成都客户'): 0.4, ('成都分仓', '成都客户'): 0.2,
    ('上海分仓', '西安客户'): 0.3, ('武汉分仓', '西安客户'): 0.5, ('成都分仓', '西安客户'): 0.3,
    ('上海分仓', '杭州客户'): 0.2, ('武汉分仓', '杭州客户'): 0.6, ('成都分仓', '杭州客户'): 0.9
}


# 客户需求数据 (单位: 件)
# 每个客户在每个周期对每种产品的需求量 (件)
demand = {
    '沈阳客户': {'橙汁': {1: 10000, 2: 12000, 3: 8000}, '苹果汁': {1: 5000, 2: 6000, 3: 7000},
               '葡萄汁': {1: 3000, 2: 4000, 3: 3500}, '芒果汁': {1: 2000, 2: 2500, 3: 2200}},
    '深圳客户': {'橙汁': {1: 15000, 2: 18000, 3: 16000}, '苹果汁': {1: 8000, 2: 9000, 3: 10000},
               '葡萄汁': {1: 6000, 2: 7000, 3: 6500}, '芒果汁': {1: 5000, 2: 5500, 3: 5200}},
    '成都客户': {'橙汁': {1: 12000, 2: 10000, 3: 14000}, '苹果汁': {1: 7000, 2: 6000, 3: 8000},
               '葡萄汁': {1: 5000, 2: 4500, 3: 5500}, '芒果汁': {1: 4000, 2: 3500, 3: 4200}},
    '西安客户': {'橙汁': {1: 8000, 2: 9000, 3: 7000}, '苹果汁': {1: 4000, 2: 4500, 3: 3800},
               '葡萄汁': {1: 3000, 2: 3200, 3: 2800}, '芒果汁': {1: 2500, 2: 2800, 3: 2400}},
    '杭州客户': {'橙汁': {1: 9000, 2: 8500, 3: 9500}, '苹果汁': {1: 5000, 2: 4800, 3: 5200},
               '葡萄汁': {1: 4000, 2: 3800, 3: 4200}, '芒果汁': {1: 3000, 2: 2800, 3: 3200}}
}


# 惩罚成本 (单位: 元/件)
# 每件未满足需求的惩罚成本 (元/件)
penalty_cost_per_unit = 100

# 地理位置数据 (用于地图可视化)
# 各个地点对应的经纬度 (纬度, 经度)
locations = {
    '北京工厂': (39.9042, 116.4074), '广州工厂': (23.1291, 113.2644), '上海工厂': (31.2304, 121.4737),
    '上海分仓': (31.2304, 121.4737), '武汉分仓': (30.5928, 114.3055), '成都分仓': (30.5728, 104.0668),
    '沈阳客户': (41.7963, 123.4315), '深圳客户': (22.5431, 114.0579), '成都客户': (30.5728, 104.0668),
    '西安客户': (34.2632, 108.9525), '杭州客户': (30.2500, 120.1600)
}

In [3]:
# Re-run Model Building and Solving cell
# --- 2. 模型构建与求解 ---

"""构建并求解供应链优化模型"""
model = LpProblem("Supply_Chain_Optimization", LpMinimize)

# 决策变量
line_on = LpVariable.dicts("Line_On", (factories, all_lines, products, periods), 0, 1, LpBinary)
produce = LpVariable.dicts("Produce", (factories, all_lines, products, periods), 0, None, LpContinuous)
flow_fw = LpVariable.dicts("Flow_FW", (factories, warehouses, products, periods), 0, None, LpContinuous)
flow_fc = LpVariable.dicts("Flow_FC", (factories, customers, products, periods), 0, None, LpContinuous)
flow_wc = LpVariable.dicts("Flow_WC", (warehouses, customers, products, periods), 0, None, LpContinuous)
factory_inventory = LpVariable.dicts("Factory_Inventory", (factories, products, periods), 0, None, LpContinuous)
warehouse_inventory = LpVariable.dicts("Warehouse_Inventory", (warehouses, products, periods), 0, None, LpContinuous)

# 目标函数：最小化总成本
total_cost = []
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if p in production_lines_info[f][line]['products']:
                    total_cost.append(production_cost[(f, line, p)] * produce[f][line][p][t])
                    total_cost.append(line_activation_cost[(f, line)] * line_on[f][line][p][t])
for f in factories:
    for p in products:
        for t in periods:
            total_cost.append(factory_inventory_cost[f][p] * factory_inventory[f][p][t])
for w in warehouses:
    for p in products:
        for t in periods:
            total_cost.append(warehouse_inventory_cost[w][p] * warehouse_inventory[w][p][t])
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_fw[(f, w)] * flow_fw[f][w][p][t])
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_fc.get((f, c), 0) * flow_fc[f][c][p][t])
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_wc.get((w, c), 0) * flow_wc[w][c][p][t])
for c in customers:
    for p in products:
        for t in periods:
            unmet_demand_var = LpVariable(f"Unmet_Demand_{c}_{p}_{t}", 0, None)
            received_flow = lpSum([flow_fc[f][c][p][t] for f in factories]) + lpSum([flow_wc[w][c][p][t] for w in warehouses])
            model += unmet_demand_var >= demand[c][p][t] - received_flow
            total_cost.append(penalty_cost_per_unit * unmet_demand_var)


model += lpSum(total_cost)

# 约束条件
for c in customers:
    for p in products:
        for t in periods:
            model += lpSum([flow_fc[f][c][p][t] for f in factories] + [flow_wc[w][c][p][t] for w in warehouses]) >= demand[c][p][t]

for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if p in production_lines_info[f][line]['products']:
                    min_val = production_lines_info[f][line]['min_daily_output'][p] * days_per_period
                    max_val = production_lines_info[f][line]['max_daily_output'][p] * days_per_period
                    model += produce[f][line][p][t] >= min_val * line_on[f][line][p][t]
                    model += produce[f][line][p][t] <= max_val * line_on[f][line][p][t]
                else:
                    model += produce[f][line][p][t] == 0
                    model += line_on[f][line][p][t] == 0

for f in factories:
    for line in production_lines_info[f]:
        for t in periods:
            model += lpSum([line_on[f][line][p][t] for p in products]) <= 1

for f in factories:
    for p in products:
        for t in periods:
            total_produced = lpSum([produce[f][line][p][t] for line in production_lines_info[f]])
            total_shipped = lpSum([flow_fw[f][w][p][t] for w in warehouses] + [flow_fc[f][c][p][t] for c in customers])
            if t == periods[0]:
                model += factory_inventory[f][p][t] == total_produced - total_shipped
            else:
                model += factory_inventory[f][p][t] == factory_inventory[f][p][t-1] + total_produced - total_shipped

for w in warehouses:
    for p in products:
        for t in periods:
            inflow = lpSum([flow_fw[f][w][p][t] for f in factories])
            outflow = lpSum([flow_wc[w][c][p][t] for c in customers])
            if t == periods[0]:
                model += warehouse_inventory[w][p][t] == inflow - outflow
            else:
                model += warehouse_inventory[w][p][t] == warehouse_inventory[w][p][t-1] + inflow - outflow

# 求解模型
model.solve()
if LpStatus[model.status] != 'Optimal':
    print("模型求解失败，未找到最优解。")

NameError: name 'LpProblem' is not defined

In [4]:
# Re-run the Production Line Utilization Visualization code
# --- 提取生产线利用率数据 ---
line_utilization_data = []
for f in factories:
    for line in production_lines_info[f]:
        for p in production_lines_info[f][line]['products']:
            for t in periods:
                produced = value(produce[f][line][p][t]) if value(produce[f][line][p][t]) is not None else 0
                max_capacity = production_lines_info[f][line]['max_daily_output'][p] * days_per_period
                if max_capacity > 0:
                    utilization = produced / max_capacity
                else:
                    utilization = 0 # Handle case where max_capacity is zero

                line_utilization_data.append({
                    '工厂': f,
                    '生产线': line,
                    '产品': p,
                    '周期（旬）': t,
                    '生产量': produced,
                    '最大产能': max_capacity,
                    '利用率': utilization
                })

line_utilization_df = pd.DataFrame(line_utilization_data)

print("\n--- 生产线利用率 ---")
if not line_utilization_df.empty:
    display(line_utilization_df)

    # 可视化生产线利用率
    fig_line_utilization = px.bar(
        line_utilization_df,
        x='周期（旬）',
        y='利用率',
        color='产品',
        facet_col='工厂',
        facet_row='生产线',
        title='生产线利用率（按工厂、生产线、产品和周期）',
        labels={'利用率': '利用率', '周期（旬）': '周期（旬）'}
    )
    fig_line_utilization.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_line_utilization.show()
else:
    print("没有可展示的生产线利用率数据。")


# --- 提取仓库空间利用率数据 (简化示例，假设最大仓储能力) ---
# 实际应用中需要更详细的仓储能力数据 (按产品、体积/重量等)
# 这里仅以总库存量占一个假设的最大容量为例

# 假设每个分仓的最大仓储能力 (单位: 件) - 这是一个简化，实际需要更精确的数据
warehouse_max_capacity = {
    '上海分仓': 100000,
    '武汉分仓': 80000,
    '成都分仓': 70000
}

warehouse_utilization_data = []
for w in warehouses:
    for t in periods:
        total_inventory = sum(
            value(warehouse_inventory[w][p][t]) if value(warehouse_inventory[w][p][t]) is not None else 0
            for p in products
        )
        max_capacity = warehouse_max_capacity.get(w, 0) # Get capacity, default to 0 if not found
        if max_capacity > 0:
            utilization = total_inventory / max_capacity
        else:
            utilization = 0 # Handle case where max_capacity is zero

        warehouse_utilization_data.append({
            '分仓': w,
            '周期（旬）': t,
            '总库存量': total_inventory,
            '最大仓储能力': max_capacity,
            '利用率': utilization
        })

warehouse_utilization_df = pd.DataFrame(warehouse_utilization_data)

print("\n--- 仓库空间利用率 ---")
if not warehouse_utilization_df.empty:
    display(warehouse_utilization_df)

    # 可视化仓库空间利用率
    fig_warehouse_utilization = px.bar(
        warehouse_utilization_df,
        x='周期（旬）',
        y='利用率',
        color='分仓',
        barmode='group',
        title='仓库空间利用率（按分仓和周期）',
        labels={'利用率': '利用率', '周期（旬）': '周期（旬）'}
    )
    fig_warehouse_utilization.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_warehouse_utilization.show()
else:
    print("没有可展示的仓库空间利用率数据。")

NameError: name 'value' is not defined

In [5]:
# Import necessary libraries, including 'value' from pulp
from pulp import *
import pandas as pd
from collections import defaultdict
import plotly.express as px
import folium
from folium.plugins import MarkerCluster, Fullscreen, PolyLineTextPath
from IPython.display import display

ModuleNotFoundError: No module named 'pulp'

In [6]:
# Re-run Data Preparation cell to ensure variables are defined and data is updated
# --- 1. 数据准备 ---

# 定义所有站点、产品和周期
factories = ['北京工厂', '广州工厂', '上海工厂'] # 工厂列表
warehouses = ['上海分仓', '武汉分仓', '成都分仓'] # 分仓列表
customers = ['沈阳客户', '深圳客户', '成都客户', '西安客户', '杭州客户'] # 客户列表
periods = [1, 2, 3]  # 三旬，每个周期代表一旬 (约10天)
days_per_period = 10 # 每旬10天
products = ['橙汁', '苹果汁', '葡萄汁', '芒果汁'] # 产品列表

# 产品数据（售价、重量、体积）
product_data = {
    # '产品名称': {'sales_price': 销售价格 (元/件), 'weight': 重量 (kg/件), 'volume': 体积 (m^3/件)}
    '橙汁': {'sales_price': 8.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '苹果汁': {'sales_price': 7.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '葡萄汁': {'sales_price': 9.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '芒果汁': {'sales_price': 10.0, 'weight': 0.5, 'volume': 0.0005} # 假设每件500ml
}

# 生产线信息
production_lines_info = {
    '北京工厂': {
        'Line_1': {
            'products': ['橙汁', '葡萄汁'], # 该生产线可以生产的产品
            'min_daily_output': {'橙汁': 2000, '葡萄汁': 1800}, # 每日最小产值 (件/天)
            'max_daily_output': {'橙汁': 5000, '葡萄汁': 4500} # 每日最大产值 (件/天)
        },
        'Line_2': {
            'products': ['苹果汁', '芒果汁'],
            'min_daily_output': {'苹果汁': 1500, '芒果汁': 1200},
            'max_daily_output': {'苹果汁': 4000, '芒果汁': 3000}
        }
    },
    '广州工厂': {
        'Line_A': {
            'products': ['橙汁', '芒果汁'],
            'min_daily_output': {'橙汁': 1500, '芒果汁': 1000},
            'max_daily_output': {'橙汁': 6000, '芒果汁': 5000}
        },
        'Line_B': {
            'products': ['苹果汁', '葡萄汁'],
            'min_daily_output': {'苹果汁': 1800, '葡萄汁': 1600},
            'max_daily_output': {'苹果汁': 6500, '葡萄汁': 5500}
        }
    },
    '上海工厂': {
        'Line_X': {
            'products': ['橙汁', '苹果汁', '葡萄汁', '芒果汁'],
            'min_daily_output': {'橙汁': 1000, '苹果汁': 800, '葡萄汁': 900, '芒果汁': 700},
            'max_daily_output': {'橙汁': 4000, '苹果汁': 3500, '葡萄汁': 3800, '芒果汁': 3000}
        }
    }
}

all_lines = list(set([line for f in factories for line in production_lines_info[f]])) # 所有生产线的集合


# 成本数据 (单位: 元/件)
# 生产成本: 每件产品在特定工厂的特定生产线上生产的成本 (元/件)
# 已将原料成本和人工成本移至此处，这里的成本现包含原料、人工及其他变动生产成本。
production_cost = {
    ('北京工厂', 'Line_1', '橙汁'): 1.5 + 1.0 + 2.0, # 原料 + 人工 + 其他变动成本
    ('北京工厂', 'Line_1', '葡萄汁'): 1.8 + 1.1 + 2.2,
    ('北京工厂', 'Line_2', '苹果汁'): 1.2 + 0.9 + 1.8,
    ('北京工厂', 'Line_2', '芒果汁'): 2.0 + 1.2 + 2.5,
    ('广州工厂', 'Line_A', '橙汁'): 1.5 + 1.0 + 1.9,
    ('广州工厂', 'Line_A', '芒果汁'): 2.0 + 1.2 + 2.4,
    ('广州工厂', 'Line_B', '苹果汁'): 1.2 + 0.9 + 1.7,
    ('广州工厂', 'Line_B', '葡萄汁'): 1.8 + 1.1 + 2.1,
    ('上海工厂', 'Line_X', '橙汁'): 1.5 + 1.0 + 2.1,
    ('上海工厂', 'Line_X', '苹果汁'): 1.2 + 0.9 + 1.9,
    ('上海工厂', 'Line_X', '葡萄汁'): 1.8 + 1.1 + 2.3,
    ('上海工厂', 'Line_X', '芒果汁'): 2.0 + 1.2 + 2.6
}

# 生产线激活成本 (单位: 元/旬)
# 每旬激活一条生产线所需的固定成本 (元/旬)
line_activation_cost = {
    ('北京工厂', 'Line_1'): 5000, ('北京工厂', 'Line_2'): 4800,
    ('广州工厂', 'Line_A'): 4500, ('广州工厂', 'Line_B'): 4000,
    ('上海工厂', 'Line_X'): 6000
}


# 仓储成本 (单位: 元/件/旬)
# 每件产品在工厂仓库中存放一旬的成本 (元/件/旬)
factory_inventory_cost = {
    '北京工厂': {'橙汁': 0.08, '苹果汁': 0.10, '葡萄汁': 0.09, '芒果汁': 0.11},
    '广州工厂': {'橙汁': 0.07, '苹果汁': 0.09, '葡萄汁': 0.08, '芒果汁': 0.10},
    '上海工厂': {'橙汁': 0.09, '苹果汁': 0.11, '葡萄汁': 0.10, '芒果汁': 0.12}
}

# 每件产品在分仓中存放一旬的成本 (元/件/旬)
warehouse_inventory_cost = {
    '上海分仓': {'橙汁': 0.1, '苹果汁': 0.12, '葡萄汁': 0.11, '芒果汁': 0.13},
    '武汉分仓': {'橙汁': 0.09, '苹果汁': 0.11, '葡萄汁': 0.10, '芒果汁': 0.12},
    '成都分仓': {'橙汁': 0.11, '苹果汁': 0.13, '葡萄汁': 0.12, '芒果汁': 0.14}
}

# 运输成本 (单位: 元/件)
# 工厂到分仓的运输成本 (元/件)
transport_cost_fw = {
    ('北京工厂', '上海分仓'): 0.5, ('北京工厂', '武汉分仓'): 0.6, ('北京工厂', '成都分仓'): 0.9,
    ('广州工厂', '上海分仓'): 0.4, ('广州工厂', '武汉分仓'): 0.3, ('广州工厂', '成都分仓'): 0.7,
    ('上海工厂', '上海分仓'): 0.1, ('上海工厂', '武汉分仓'): 0.5, ('上海工厂', '成都分仓'): 0.8
}
# 工厂到客户的运输成本 (元/件)
transport_cost_fc = {
    ('北京工厂', '沈阳客户'): 0.2, ('广州工厂', '沈阳客户'): 1.5, ('上海工厂', '沈阳客户'): 0.8,
    ('北京工厂', '深圳客户'): 1.8, ('广州工厂', '深圳客户'): 0.2, ('上海工厂', '深圳客户'): 0.7,
    ('北京工厂', '成都客户'): 1.2, ('广州工厂', '成都客户'): 0.9, ('上海工厂', '成都客户'): 0.4,
    ('北京工厂', '西安客户'): 0.7, ('广州工厂', '西安客户'): 1.1, ('上海工厂', '西安客户'): 0.6,
    ('北京工厂', '杭州客户'): 0.9, ('广州工厂', '杭州客户'): 0.8, ('上海工厂', '杭州客户'): 0.3
}
# 分仓到客户的运输成本 (元/件)
transport_cost_wc = {
    ('上海分仓', '沈阳客户'): 0.8, ('武汉分仓', '沈阳客户'): 1.3, ('成都分仓', '沈阳客户'): 1.5,
    ('上海分仓', '深圳客户'): 0.6, ('武汉分仓', '深圳客户'): 0.7, ('成都分仓', '成都客户'): 1.0,
    ('上海分仓', '成都客户'): 0.5, ('武汉分仓', '成都客户'): 0.4, ('成都分仓', '成都客户'): 0.2,
    ('上海分仓', '西安客户'): 0.3, ('武汉分仓', '西安客户'): 0.5, ('成都分仓', '西安客户'): 0.3,
    ('上海分仓', '杭州客户'): 0.2, ('武汉分仓', '杭州客户'): 0.6, ('成都分仓', '杭州客户'): 0.9
}


# 客户需求数据 (单位: 件)
# 每个客户在每个周期对每种产品的需求量 (件)
demand = {
    '沈阳客户': {'橙汁': {1: 10000, 2: 12000, 3: 8000}, '苹果汁': {1: 5000, 2: 6000, 3: 7000},
               '葡萄汁': {1: 3000, 2: 4000, 3: 3500}, '芒果汁': {1: 2000, 2: 2500, 3: 2200}},
    '深圳客户': {'橙汁': {1: 15000, 2: 18000, 3: 16000}, '苹果汁': {1: 8000, 2: 9000, 3: 10000},
               '葡萄汁': {1: 6000, 2: 7000, 3: 6500}, '芒果汁': {1: 5000, 2: 5500, 3: 5200}},
    '成都客户': {'橙汁': {1: 12000, 2: 10000, 3: 14000}, '苹果汁': {1: 7000, 2: 6000, 3: 8000},
               '葡萄汁': {1: 5000, 2: 4500, 3: 5500}, '芒果汁': {1: 4000, 2: 3500, 3: 4200}},
    '西安客户': {'橙汁': {1: 8000, 2: 9000, 3: 7000}, '苹果汁': {1: 4000, 2: 4500, 3: 3800},
               '葡萄汁': {1: 3000, 2: 3200, 3: 2800}, '芒果汁': {1: 2500, 2: 2800, 3: 2400}},
    '杭州客户': {'橙汁': {1: 9000, 2: 8500, 3: 9500}, '苹果汁': {1: 5000, 2: 4800, 3: 5200},
               '葡萄汁': {1: 4000, 2: 3800, 3: 4200}, '芒果汁': {1: 3000, 2: 2800, 3: 3200}}
}


# 惩罚成本 (单位: 元/件)
# 每件未满足需求的惩罚成本 (元/件)
penalty_cost_per_unit = 100

# 地理位置数据 (用于地图可视化)
# 各个地点对应的经纬度 (纬度, 经度)
locations = {
    '北京工厂': (39.9042, 116.4074), '广州工厂': (23.1291, 113.2644), '上海工厂': (31.2304, 121.4737),
    '上海分仓': (31.2304, 121.4737), '武汉分仓': (30.5928, 114.3055), '成都分仓': (30.5728, 104.0668),
    '沈阳客户': (41.7963, 123.4315), '深圳客户': (22.5431, 114.0579), '成都客户': (30.5728, 104.0668),
    '西安客户': (34.2632, 108.9525), '杭州客户': (30.2500, 120.1600)
}

In [7]:
# Re-run Model Building and Solving cell
# --- 2. 模型构建与求解 ---

"""构建并求解供应链优化模型"""
model = LpProblem("Supply_Chain_Optimization", LpMinimize)

# 决策变量
line_on = LpVariable.dicts("Line_On", (factories, all_lines, products, periods), 0, 1, LpBinary)
produce = LpVariable.dicts("Produce", (factories, all_lines, products, periods), 0, None, LpContinuous)
flow_fw = LpVariable.dicts("Flow_FW", (factories, warehouses, products, periods), 0, None, LpContinuous)
flow_fc = LpVariable.dicts("Flow_FC", (factories, customers, products, periods), 0, None, LpContinuous)
flow_wc = LpVariable.dicts("Flow_WC", (warehouses, customers, products, periods), 0, None, LpContinuous)
factory_inventory = LpVariable.dicts("Factory_Inventory", (factories, products, periods), 0, None, LpContinuous)
warehouse_inventory = LpVariable.dicts("Warehouse_Inventory", (warehouses, products, periods), 0, None, LpContinuous)

# 目标函数：最小化总成本
total_cost = []
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if p in production_lines_info[f][line]['products']:
                    total_cost.append(production_cost[(f, line, p)] * produce[f][line][p][t])
                    total_cost.append(line_activation_cost[(f, line)] * line_on[f][line][p][t])
for f in factories:
    for p in products:
        for t in periods:
            total_cost.append(factory_inventory_cost[f][p] * factory_inventory[f][p][t])
for w in warehouses:
    for p in products:
        for t in periods:
            total_cost.append(warehouse_inventory_cost[w][p] * warehouse_inventory[w][p][t])
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_fw[(f, w)] * flow_fw[f][w][p][t])
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_fc.get((f, c), 0) * flow_fc[f][c][p][t])
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_wc.get((w, c), 0) * flow_wc[w][c][p][t])
for c in customers:
    for p in products:
        for t in periods:
            unmet_demand_var = LpVariable(f"Unmet_Demand_{c}_{p}_{t}", 0, None)
            received_flow = lpSum([flow_fc[f][c][p][t] for f in factories]) + lpSum([flow_wc[w][c][p][t] for w in warehouses])
            model += unmet_demand_var >= demand[c][p][t] - received_flow
            total_cost.append(penalty_cost_per_unit * unmet_demand_var)


model += lpSum(total_cost)

# 约束条件
for c in customers:
    for p in products:
        for t in periods:
            model += lpSum([flow_fc[f][c][p][t] for f in factories] + [flow_wc[w][c][p][t] for w in warehouses]) >= demand[c][p][t]

for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if p in production_lines_info[f][line]['products']:
                    min_val = production_lines_info[f][line]['min_daily_output'][p] * days_per_period
                    max_val = production_lines_info[f][line]['max_daily_output'][p] * days_per_period
                    model += produce[f][line][p][t] >= min_val * line_on[f][line][p][t]
                    model += produce[f][line][p][t] <= max_val * line_on[f][line][p][t]
                else:
                    model += produce[f][line][p][t] == 0
                    model += line_on[f][line][p][t] == 0

for f in factories:
    for line in production_lines_info[f]:
        for t in periods:
            model += lpSum([line_on[f][line][p][t] for p in products]) <= 1

for f in factories:
    for p in products:
        for t in periods:
            total_produced = lpSum([produce[f][line][p][t] for line in production_lines_info[f]])
            total_shipped = lpSum([flow_fw[f][w][p][t] for w in warehouses] + [flow_fc[f][c][p][t] for c in customers])
            if t == periods[0]:
                model += factory_inventory[f][p][t] == total_produced - total_shipped
            else:
                model += factory_inventory[f][p][t] == factory_inventory[f][p][t-1] + total_produced - total_shipped

for w in warehouses:
    for p in products:
        for t in periods:
            inflow = lpSum([flow_fw[f][w][p][t] for f in factories])
            outflow = lpSum([flow_wc[w][c][p][t] for c in customers])
            if t == periods[0]:
                model += warehouse_inventory[w][p][t] == inflow - outflow
            else:
                model += warehouse_inventory[w][p][t] == warehouse_inventory[w][p][t-1] + inflow - outflow

# 求解模型
model.solve()
if LpStatus[model.status] != 'Optimal':
    print("模型求解失败，未找到最优解。")

NameError: name 'LpProblem' is not defined

In [8]:
# Re-run the Production Line Utilization Visualization code
# --- 提取生产线利用率数据 ---
line_utilization_data = []
for f in factories:
    for line in production_lines_info[f]:
        for p in production_lines_info[f][line]['products']:
            for t in periods:
                produced = value(produce[f][line][p][t]) if value(produce[f][line][p][t]) is not None else 0
                max_capacity = production_lines_info[f][line]['max_daily_output'][p] * days_per_period
                if max_capacity > 0:
                    utilization = produced / max_capacity
                else:
                    utilization = 0 # Handle case where max_capacity is zero

                line_utilization_data.append({
                    '工厂': f,
                    '生产线': line,
                    '产品': p,
                    '周期（旬）': t,
                    '生产量': produced,
                    '最大产能': max_capacity,
                    '利用率': utilization
                })

line_utilization_df = pd.DataFrame(line_utilization_data)

print("\n--- 生产线利用率 ---")
if not line_utilization_df.empty:
    display(line_utilization_df)

    # 可视化生产线利用率
    fig_line_utilization = px.bar(
        line_utilization_df,
        x='周期（旬）',
        y='利用率',
        color='产品',
        facet_col='工厂',
        facet_row='生产线',
        title='生产线利用率（按工厂、生产线、产品和周期）',
        labels={'利用率': '利用率', '周期（旬）': '周期（旬）'}
    )
    fig_line_utilization.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_line_utilization.show()
else:
    print("没有可展示的生产线利用率数据。")


# --- 提取仓库空间利用率数据 (简化示例，假设最大仓储能力) ---
# 实际应用中需要更详细的仓储能力数据 (按产品、体积/重量等)
# 这里仅以总库存量占一个假设的最大容量为例

# 假设每个分仓的最大仓储能力 (单位: 件) - 这是一个简化，实际需要更精确的数据
warehouse_max_capacity = {
    '上海分仓': 100000,
    '武汉分仓': 80000,
    '成都分仓': 70000
}

warehouse_utilization_data = []
for w in warehouses:
    for t in periods:
        total_inventory = sum(
            value(warehouse_inventory[w][p][t]) if value(warehouse_inventory[w][p][t]) is not None else 0
            for p in products
        )
        max_capacity = warehouse_max_capacity.get(w, 0) # Get capacity, default to 0 if not found
        if max_capacity > 0:
            utilization = total_inventory / max_capacity
        else:
            utilization = 0 # Handle case where max_capacity is zero

        warehouse_utilization_data.append({
            '分仓': w,
            '周期（旬）': t,
            '总库存量': total_inventory,
            '最大仓储能力': max_capacity,
            '利用率': utilization
        })

warehouse_utilization_df = pd.DataFrame(warehouse_utilization_data)

print("\n--- 仓库空间利用率 ---")
if not warehouse_utilization_df.empty:
    display(warehouse_utilization_df)

    # 可视化仓库空间利用率
    fig_warehouse_utilization = px.bar(
        warehouse_utilization_df,
        x='周期（旬）',
        y='利用率',
        color='分仓',
        barmode='group',
        title='仓库空间利用率（按分仓和周期）',
        labels={'利用率': '利用率', '周期（旬）': '周期（旬）'}
    )
    fig_warehouse_utilization.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_warehouse_utilization.show()
else:
    print("没有可展示的仓库空间利用率数据。")

NameError: name 'value' is not defined

In [9]:
# Import necessary libraries, including 'value' from pulp
from pulp import *
import pandas as pd
from collections import defaultdict
import plotly.express as px
import folium
from folium.plugins import MarkerCluster, Fullscreen, PolyLineTextPath
from IPython.display import display

ModuleNotFoundError: No module named 'pulp'

In [10]:
# Re-run Data Preparation cell to ensure variables are defined and data is updated
# --- 1. 数据准备 ---

# 定义所有站点、产品和周期
factories = ['北京工厂', '广州工厂', '上海工厂'] # 工厂列表
warehouses = ['上海分仓', '武汉分仓', '成都分仓'] # 分仓列表
customers = ['沈阳客户', '深圳客户', '成都客户', '西安客户', '杭州客户'] # 客户列表
periods = [1, 2, 3]  # 三旬，每个周期代表一旬 (约10天)
days_per_period = 10 # 每旬10天
products = ['橙汁', '苹果汁', '葡萄汁', '芒果汁'] # 产品列表

# 产品数据（售价、重量、体积）
product_data = {
    # '产品名称': {'sales_price': 销售价格 (元/件), 'weight': 重量 (kg/件), 'volume': 体积 (m^3/件)}
    '橙汁': {'sales_price': 8.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '苹果汁': {'sales_price': 7.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '葡萄汁': {'sales_price': 9.0, 'weight': 0.5, 'volume': 0.0005}, # 假设每件500ml
    '芒果汁': {'sales_price': 10.0, 'weight': 0.5, 'volume': 0.0005} # 假设每件500ml
}

# 生产线信息
production_lines_info = {
    '北京工厂': {
        'Line_1': {
            'products': ['橙汁', '葡萄汁'], # 该生产线可以生产的产品
            'min_daily_output': {'橙汁': 2000, '葡萄汁': 1800}, # 每日最小产值 (件/天)
            'max_daily_output': {'橙汁': 5000, '葡萄汁': 4500} # 每日最大产值 (件/天)
        },
        'Line_2': {
            'products': ['苹果汁', '芒果汁'],
            'min_daily_output': {'苹果汁': 1500, '芒果汁': 1200},
            'max_daily_output': {'苹果汁': 4000, '芒果汁': 3000}
        }
    },
    '广州工厂': {
        'Line_A': {
            'products': ['橙汁', '芒果汁'],
            'min_daily_output': {'橙汁': 1500, '芒果汁': 1000},
            'max_daily_output': {'橙汁': 6000, '芒果汁': 5000}
        },
        'Line_B': {
            'products': ['苹果汁', '葡萄汁'],
            'min_daily_output': {'苹果汁': 1800, '葡萄汁': 1600},
            'max_daily_output': {'苹果汁': 6500, '葡萄汁': 5500}
        }
    },
    '上海工厂': {
        'Line_X': {
            'products': ['橙汁', '苹果汁', '葡萄汁', '芒果汁'],
            'min_daily_output': {'橙汁': 1000, '苹果汁': 800, '葡萄汁': 900, '芒果汁': 700},
            'max_daily_output': {'橙汁': 4000, '苹果汁': 3500, '葡萄汁': 3800, '芒果汁': 3000}
        }
    }
}

all_lines = list(set([line for f in factories for line in production_lines_info[f]])) # 所有生产线的集合


# 成本数据 (单位: 元/件)
# 生产成本: 每件产品在特定工厂的特定生产线上生产的成本 (元/件)
# 已将原料成本和人工成本移至此处，这里的成本现包含原料、人工及其他变动生产成本。
production_cost = {
    ('北京工厂', 'Line_1', '橙汁'): 1.5 + 1.0 + 2.0, # 原料 + 人工 + 其他变动成本
    ('北京工厂', 'Line_1', '葡萄汁'): 1.8 + 1.1 + 2.2,
    ('北京工厂', 'Line_2', '苹果汁'): 1.2 + 0.9 + 1.8,
    ('北京工厂', 'Line_2', '芒果汁'): 2.0 + 1.2 + 2.5,
    ('广州工厂', 'Line_A', '橙汁'): 1.5 + 1.0 + 1.9,
    ('广州工厂', 'Line_A', '芒果汁'): 2.0 + 1.2 + 2.4,
    ('广州工厂', 'Line_B', '苹果汁'): 1.2 + 0.9 + 1.7,
    ('广州工厂', 'Line_B', '葡萄汁'): 1.8 + 1.1 + 2.1,
    ('上海工厂', 'Line_X', '橙汁'): 1.5 + 1.0 + 2.1,
    ('上海工厂', 'Line_X', '苹果汁'): 1.2 + 0.9 + 1.9,
    ('上海工厂', 'Line_X', '葡萄汁'): 1.8 + 1.1 + 2.3,
    ('上海工厂', 'Line_X', '芒果汁'): 2.0 + 1.2 + 2.6
}

# 生产线激活成本 (单位: 元/旬)
# 每旬激活一条生产线所需的固定成本 (元/旬)
line_activation_cost = {
    ('北京工厂', 'Line_1'): 5000, ('北京工厂', 'Line_2'): 4800,
    ('广州工厂', 'Line_A'): 4500, ('广州工厂', 'Line_B'): 4000,
    ('上海工厂', 'Line_X'): 6000
}


# 仓储成本 (单位: 元/件/旬)
# 每件产品在工厂仓库中存放一旬的成本 (元/件/旬)
factory_inventory_cost = {
    '北京工厂': {'橙汁': 0.08, '苹果汁': 0.10, '葡萄汁': 0.09, '芒果汁': 0.11},
    '广州工厂': {'橙汁': 0.07, '苹果汁': 0.09, '葡萄汁': 0.08, '芒果汁': 0.10},
    '上海工厂': {'橙汁': 0.09, '苹果汁': 0.11, '葡萄汁': 0.10, '芒果汁': 0.12}
}

# 每件产品在分仓中存放一旬的成本 (元/件/旬)
warehouse_inventory_cost = {
    '上海分仓': {'橙汁': 0.1, '苹果汁': 0.12, '葡萄汁': 0.11, '芒果汁': 0.13},
    '武汉分仓': {'橙汁': 0.09, '苹果汁': 0.11, '葡萄汁': 0.10, '芒果汁': 0.12},
    '成都分仓': {'橙汁': 0.11, '苹果汁': 0.13, '葡萄汁': 0.12, '芒果汁': 0.14}
}

# 运输成本 (单位: 元/件)
# 工厂到分仓的运输成本 (元/件)
transport_cost_fw = {
    ('北京工厂', '上海分仓'): 0.5, ('北京工厂', '武汉分仓'): 0.6, ('北京工厂', '成都分仓'): 0.9,
    ('广州工厂', '上海分仓'): 0.4, ('广州工厂', '武汉分仓'): 0.3, ('广州工厂', '成都分仓'): 0.7,
    ('上海工厂', '上海分仓'): 0.1, ('上海工厂', '武汉分仓'): 0.5, ('上海工厂', '成都分仓'): 0.8
}
# 工厂到客户的运输成本 (元/件)
transport_cost_fc = {
    ('北京工厂', '沈阳客户'): 0.2, ('广州工厂', '沈阳客户'): 1.5, ('上海工厂', '沈阳客户'): 0.8,
    ('北京工厂', '深圳客户'): 1.8, ('广州工厂', '深圳客户'): 0.2, ('上海工厂', '深圳客户'): 0.7,
    ('北京工厂', '成都客户'): 1.2, ('广州工厂', '成都客户'): 0.9, ('上海工厂', '成都客户'): 0.4,
    ('北京工厂', '西安客户'): 0.7, ('广州工厂', '西安客户'): 1.1, ('上海工厂', '西安客户'): 0.6,
    ('北京工厂', '杭州客户'): 0.9, ('广州工厂', '杭州客户'): 0.8, ('上海工厂', '杭州客户'): 0.3
}
# 分仓到客户的运输成本 (元/件)
transport_cost_wc = {
    ('上海分仓', '沈阳客户'): 0.8, ('武汉分仓', '沈阳客户'): 1.3, ('成都分仓', '沈阳客户'): 1.5,
    ('上海分仓', '深圳客户'): 0.6, ('武汉分仓', '深圳客户'): 0.7, ('成都分仓', '成都客户'): 1.0,
    ('上海分仓', '成都客户'): 0.5, ('武汉分仓', '成都客户'): 0.4, ('成都分仓', '成都客户'): 0.2,
    ('上海分仓', '西安客户'): 0.3, ('武汉分仓', '西安客户'): 0.5, ('成都分仓', '西安客户'): 0.3,
    ('上海分仓', '杭州客户'): 0.2, ('武汉分仓', '杭州客户'): 0.6, ('成都分仓', '杭州客户'): 0.9
}


# 客户需求数据 (单位: 件)
# 每个客户在每个周期对每种产品的需求量 (件)
demand = {
    '沈阳客户': {'橙汁': {1: 10000, 2: 12000, 3: 8000}, '苹果汁': {1: 5000, 2: 6000, 3: 7000},
               '葡萄汁': {1: 3000, 2: 4000, 3: 3500}, '芒果汁': {1: 2000, 2: 2500, 3: 2200}},
    '深圳客户': {'橙汁': {1: 15000, 2: 18000, 3: 16000}, '苹果汁': {1: 8000, 2: 9000, 3: 10000},
               '葡萄汁': {1: 6000, 2: 7000, 3: 6500}, '芒果汁': {1: 5000, 2: 5500, 3: 5200}},
    '成都客户': {'橙汁': {1: 12000, 2: 10000, 3: 14000}, '苹果汁': {1: 7000, 2: 6000, 3: 8000},
               '葡萄汁': {1: 5000, 2: 4500, 3: 5500}, '芒果汁': {1: 4000, 2: 3500, 3: 4200}},
    '西安客户': {'橙汁': {1: 8000, 2: 9000, 3: 7000}, '苹果汁': {1: 4000, 2: 4500, 3: 3800},
               '葡萄汁': {1: 3000, 2: 3200, 3: 2800}, '芒果汁': {1: 2500, 2: 2800, 3: 2400}},
    '杭州客户': {'橙汁': {1: 9000, 2: 8500, 3: 9500}, '苹果汁': {1: 5000, 2: 4800, 3: 5200},
               '葡萄汁': {1: 4000, 2: 3800, 3: 4200}, '芒果汁': {1: 3000, 2: 2800, 3: 3200}}
}


# 惩罚成本 (单位: 元/件)
# 每件未满足需求的惩罚成本 (元/件)
penalty_cost_per_unit = 100

# 地理位置数据 (用于地图可视化)
# 各个地点对应的经纬度 (纬度, 经度)
locations = {
    '北京工厂': (39.9042, 116.4074), '广州工厂': (23.1291, 113.2644), '上海工厂': (31.2304, 121.4737),
    '上海分仓': (31.2304, 121.4737), '武汉分仓': (30.5928, 114.3055), '成都分仓': (30.5728, 104.0668),
    '沈阳客户': (41.7963, 123.4315), '深圳客户': (22.5431, 114.0579), '成都客户': (30.5728, 104.0668),
    '西安客户': (34.2632, 108.9525), '杭州客户': (30.2500, 120.1600)
}

In [11]:
# Re-run Model Building and Solving cell
# --- 2. 模型构建与求解 ---

"""构建并求解供应链优化模型"""
model = LpProblem("Supply_Chain_Optimization", LpMinimize)

# 决策变量
line_on = LpVariable.dicts("Line_On", (factories, all_lines, products, periods), 0, 1, LpBinary)
produce = LpVariable.dicts("Produce", (factories, all_lines, products, periods), 0, None, LpContinuous)
flow_fw = LpVariable.dicts("Flow_FW", (factories, warehouses, products, periods), 0, None, LpContinuous)
flow_fc = LpVariable.dicts("Flow_FC", (factories, customers, products, periods), 0, None, LpContinuous)
flow_wc = LpVariable.dicts("Flow_WC", (warehouses, customers, products, periods), 0, None, LpContinuous)
factory_inventory = LpVariable.dicts("Factory_Inventory", (factories, products, periods), 0, None, LpContinuous)
warehouse_inventory = LpVariable.dicts("Warehouse_Inventory", (warehouses, products, periods), 0, None, LpContinuous)

# 目标函数：最小化总成本
total_cost = []
for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if p in production_lines_info[f][line]['products']:
                    total_cost.append(production_cost[(f, line, p)] * produce[f][line][p][t])
                    total_cost.append(line_activation_cost[(f, line)] * line_on[f][line][p][t])
for f in factories:
    for p in products:
        for t in periods:
            total_cost.append(factory_inventory_cost[f][p] * factory_inventory[f][p][t])
for w in warehouses:
    for p in products:
        for t in periods:
            total_cost.append(warehouse_inventory_cost[w][p] * warehouse_inventory[w][p][t])
for f in factories:
    for w in warehouses:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_fw[(f, w)] * flow_fw[f][w][p][t])
for f in factories:
    for c in customers:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_fc.get((f, c), 0) * flow_fc[f][c][p][t])
for w in warehouses:
    for c in customers:
        for p in products:
            for t in periods:
                total_cost.append(transport_cost_wc.get((w, c), 0) * flow_wc[w][c][p][t])
for c in customers:
    for p in products:
        for t in periods:
            unmet_demand_var = LpVariable(f"Unmet_Demand_{c}_{p}_{t}", 0, None)
            received_flow = lpSum([flow_fc[f][c][p][t] for f in factories]) + lpSum([flow_wc[w][c][p][t] for w in warehouses])
            model += unmet_demand_var >= demand[c][p][t] - received_flow
            total_cost.append(penalty_cost_per_unit * unmet_demand_var)


model += lpSum(total_cost)

# 约束条件
for c in customers:
    for p in products:
        for t in periods:
            model += lpSum([flow_fc[f][c][p][t] for f in factories] + [flow_wc[w][c][p][t] for w in warehouses]) >= demand[c][p][t]

for f in factories:
    for line in production_lines_info[f]:
        for p in products:
            for t in periods:
                if p in production_lines_info[f][line]['products']:
                    min_val = production_lines_info[f][line]['min_daily_output'][p] * days_per_period
                    max_val = production_lines_info[f][line]['max_daily_output'][p] * days_per_period
                    model += produce[f][line][p][t] >= min_val * line_on[f][line][p][t]
                    model += produce[f][line][p][t] <= max_val * line_on[f][line][p][t]
                else:
                    model += produce[f][line][p][t] == 0
                    model += line_on[f][line][p][t] == 0

for f in factories:
    for line in production_lines_info[f]:
        for t in periods:
            model += lpSum([line_on[f][line][p][t] for p in products]) <= 1

for f in factories:
    for p in products:
        for t in periods:
            total_produced = lpSum([produce[f][line][p][t] for line in production_lines_info[f]])
            total_shipped = lpSum([flow_fw[f][w][p][t] for w in warehouses] + [flow_fc[f][c][p][t] for c in customers])
            if t == periods[0]:
                model += factory_inventory[f][p][t] == total_produced - total_shipped
            else:
                model += factory_inventory[f][p][t] == factory_inventory[f][p][t-1] + total_produced - total_shipped

for w in warehouses:
    for p in products:
        for t in periods:
            inflow = lpSum([flow_fw[f][w][p][t] for f in factories])
            outflow = lpSum([flow_wc[w][c][p][t] for c in customers])
            if t == periods[0]:
                model += warehouse_inventory[w][p][t] == inflow - outflow
            else:
                model += warehouse_inventory[w][p][t] == warehouse_inventory[w][p][t-1] + inflow - outflow

# 求解模型
model.solve()
if LpStatus[model.status] != 'Optimal':
    print("模型求解失败，未找到最优解。")

NameError: name 'LpProblem' is not defined

In [12]:
# Re-run the Production Line Utilization Visualization code
# --- 提取生产线利用率数据 ---
line_utilization_data = []
for f in factories:
    for line in production_lines_info[f]:
        for p in production_lines_info[f][line]['products']:
            for t in periods:
                produced = value(produce[f][line][p][t]) if value(produce[f][line][p][t]) is not None else 0
                max_capacity = production_lines_info[f][line]['max_daily_output'][p] * days_per_period
                if max_capacity > 0:
                    utilization = produced / max_capacity
                else:
                    utilization = 0 # Handle case where max_capacity is zero

                line_utilization_data.append({
                    '工厂': f,
                    '生产线': line,
                    '产品': p,
                    '周期（旬）': t,
                    '生产量': produced,
                    '最大产能': max_capacity,
                    '利用率': utilization
                })

line_utilization_df = pd.DataFrame(line_utilization_data)

print("\n--- 生产线利用率 ---")
if not line_utilization_df.empty:
    display(line_utilization_df)

    # 可视化生产线利用率
    fig_line_utilization = px.bar(
        line_utilization_df,
        x='周期（旬）',
        y='利用率',
        color='产品',
        facet_col='工厂',
        facet_row='生产线',
        title='生产线利用率（按工厂、生产线、产品和周期）',
        labels={'利用率': '利用率', '周期（旬）': '周期（旬）'}
    )
    fig_line_utilization.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_line_utilization.show()
else:
    print("没有可展示的生产线利用率数据。")


# --- 提取仓库空间利用率数据 (简化示例，假设最大仓储能力) ---
# 实际应用中需要更详细的仓储能力数据 (按产品、体积/重量等)
# 这里仅以总库存量占一个假设的最大容量为例

# 假设每个分仓的最大仓储能力 (单位: 件) - 这是一个简化，实际需要更精确的数据
warehouse_max_capacity = {
    '上海分仓': 100000,
    '武汉分仓': 80000,
    '成都分仓': 70000
}

warehouse_utilization_data = []
for w in warehouses:
    for t in periods:
        total_inventory = sum(
            value(warehouse_inventory[w][p][t]) if value(warehouse_inventory[w][p][t]) is not None else 0
            for p in products
        )
        max_capacity = warehouse_max_capacity.get(w, 0) # Get capacity, default to 0 if not found
        if max_capacity > 0:
            utilization = total_inventory / max_capacity
        else:
            utilization = 0 # Handle case where max_capacity is zero

        warehouse_utilization_data.append({
            '分仓': w,
            '周期（旬）': t,
            '总库存量': total_inventory,
            '最大仓储能力': max_capacity,
            '利用率': utilization
        })

warehouse_utilization_df = pd.DataFrame(warehouse_utilization_data)

print("\n--- 仓库空间利用率 ---")
if not warehouse_utilization_df.empty:
    display(warehouse_utilization_df)

    # 可视化仓库空间利用率
    fig_warehouse_utilization = px.bar(
        warehouse_utilization_df,
        x='周期（旬）',
        y='利用率',
        color='分仓',
        barmode='group',
        title='仓库空间利用率（按分仓和周期）',
        labels={'利用率': '利用率', '周期（旬）': '周期（旬）'}
    )
    fig_warehouse_utilization.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_warehouse_utilization.show()
else:
    print("没有可展示的仓库空间利用率数据。")

NameError: name 'value' is not defined

In [24]:
# Re-run the Calculate Unit Production Cost cell
# --- 提取单位生产成本 ---

production_cost_per_unit_data = []
for f in factories:
    for line in production_lines_info[f]:
        for p in production_lines_info[f][line]['products']:
            cost = production_cost.get((f, line, p), 0)
            production_cost_per_unit_data.append({
                '工厂': f,
                '生产线': line,
                '产品': p,
                '单位生产成本': cost
            })

production_cost_per_unit_df = pd.DataFrame(production_cost_per_unit_data)

print("\n--- 单位生产成本 (按工厂、生产线、产品) ---")
if not production_cost_per_unit_df.empty:
    display(production_cost_per_unit_df)
else:
    print("未能提取单位生产成本数据。")


--- 单位生产成本 (按工厂、生产线、产品) ---


,工厂,生产线,产品,单位生产成本
0,北京工厂,Line_1,橙汁,4.5
1,北京工厂,Line_1,葡萄汁,5.1
2,北京工厂,Line_2,苹果汁,3.9
3,北京工厂,Line_2,芒果汁,5.7
4,广州工厂,Line_A,橙汁,4.4
5,广州工厂,Line_A,芒果汁,5.6
6,广州工厂,Line_B,苹果汁,3.8
7,广州工厂,Line_B,葡萄汁,5.0
8,上海工厂,Line_X,橙汁,4.6
9,上海工厂,Line_X,苹果汁,4.0


In [25]:
# Re-run the Calculate Unit Contribution Margin (Production Cost Only) cell
# --- 计算单位边际贡献（仅考虑生产成本） ---

# Merge unit production cost with sales price data
unit_contribution_margin_prod_cost_data = []
for index, row in production_cost_per_unit_df.iterrows():
    product = row['产品']
    factory = row['工厂']
    line = row['生产线']
    unit_production_cost = row['单位生产成本']

    sales_price = product_data[product]['sales_price']
    unit_contribution_margin_prod_cost = sales_price - unit_production_cost

    unit_contribution_margin_prod_cost_data.append({
        '工厂': factory,
        '生产线': line,
        '产品': product,
        '单位生产成本': unit_production_cost,
        '销售价格': sales_price,
        '单位边际贡献 (仅生产成本)': unit_contribution_margin_prod_cost
    })

unit_contribution_margin_prod_cost_df = pd.DataFrame(unit_contribution_margin_prod_cost_data)

print("\n--- 单位边际贡献 (仅考虑生产成本，按工厂、生产线、产品) ---")
if not unit_contribution_margin_prod_cost_df.empty:
    display(unit_contribution_margin_prod_cost_df)
else:
    print("未能计算单位边际贡献（仅考虑生产成本）。")


--- 单位边际贡献 (仅考虑生产成本，按工厂、生产线、产品) ---


,工厂,生产线,产品,单位生产成本,销售价格,单位边际贡献 (仅生产成本)
0,北京工厂,Line_1,橙汁,4.5,8.0,3.5
1,北京工厂,Line_1,葡萄汁,5.1,9.0,3.9
2,北京工厂,Line_2,苹果汁,3.9,7.0,3.1
3,北京工厂,Line_2,芒果汁,5.7,10.0,4.3
4,广州工厂,Line_A,橙汁,4.4,8.0,3.6
5,广州工厂,Line_A,芒果汁,5.6,10.0,4.4
6,广州工厂,Line_B,苹果汁,3.8,7.0,3.2
7,广州工厂,Line_B,葡萄汁,5.0,9.0,4.0
8,上海工厂,Line_X,橙汁,4.6,8.0,3.4
9,上海工厂,Line_X,苹果汁,4.0,7.0,3.0


In [26]:
# Re-run the Calculate Break-even Quantity (Production and Activation Cost Only) cell
# --- 计算盈亏平衡产量（仅考虑生产成本和激活成本） ---

# We need to calculate the break-even quantity per production line per period,
# considering only the line activation cost (fixed) and the contribution margin based on production cost (variable).

break_even_prod_act_cost_data = []

# Iterate through each production line and period where production occurred
# We need to find the activated lines in each period
activated_lines_per_period = defaultdict(set)
for f in factories:
    for line in production_lines_info[f]:
        for t in periods:
            # Check if the line was "on" in this period based on the model results
            # We assume a line is "on" if any product was produced on it in that period
            total_produced_on_line_in_period = sum(
                 value(produce[f][line][p][t]) if value(produce[f][line][p][t]) is not None else 0
                 for p in production_lines_info[f][line]['products']
            )
            if total_produced_on_line_in_period > 0:
                 activated_lines_per_period[(f, line, t)].add(line)


for (factory, line, period) in activated_lines_per_period:
    # Get the activation cost for this line and period
    activation_cost = line_activation_cost.get((factory, line), 0)

    # We need the average unit contribution margin (only production cost) for the products produced on this line in this period.
    # This is an approximation as different products on the same line have different contribution margins.
    # A more precise method would involve the exact mix of products produced.
    # For this simplified calculation, we'll average the unit contribution margin for all products that *can* be produced on this line.

    # Filter the unit_contribution_margin_prod_cost_df for products produced on this line
    prods_on_this_line = production_lines_info[factory][line]['products']
    cm_for_this_line_prods = unit_contribution_margin_prod_cost_df[
        (unit_contribution_margin_prod_cost_df['工厂'] == factory) &
        (unit_contribution_margin_prod_cost_df['生产线'] == line) &
        (unit_contribution_margin_prod_cost_df['产品'].isin(prods_on_this_line))
    ]

    average_unit_cm = cm_for_this_line_prods['单位边际贡献 (仅生产成本)'].mean() if not cm_for_this_line_prods.empty else 0

    # Calculate break-even quantity
    epsilon = 1e-9 # Add epsilon for safety
    break_even_quantity = activation_cost / (average_unit_cm + epsilon) if average_unit_cm > 0 else float('inf')

    break_even_prod_act_cost_data.append({
        '工厂': factory,
        '生产线': line,
        '周期（旬）': period,
        '生产线激活成本': activation_cost,
        '平均单位边际贡献 (仅生产成本)': average_unit_cm,
        '盈亏平衡产量 (件, 仅生产+激活成本)': break_even_quantity
    })

break_even_prod_act_cost_df = pd.DataFrame(break_even_prod_act_cost_data)

print("\n--- 生产线盈亏平衡产量 (仅考虑生产+激活成本, 按工厂、生产线、周期) ---")
if not break_even_prod_act_cost_df.empty:
    display(break_even_prod_act_cost_df)
else:
    print("未能计算生产线盈亏平衡产量（仅考虑生产+激活成本）。")

# Summarize by Factory (sum of activation costs, weighted average CM)
break_even_factory_prod_act_cost_data = []
for f in factories:
    factory_lines = [line for line in production_lines_info[f]]
    total_activation_cost_factory = sum(line_activation_cost.get((f, line), 0) for line in factory_lines)

    # Calculate weighted average unit CM for the factory based on total production volume
    # Need total production volume and total contribution margin (production cost only) for the factory
    factory_production_for_cm = production_plan_with_cm[production_plan_with_cm['工厂'] == f].copy() # Use a copy to avoid SettingWithCopyWarning
    if not factory_production_for_cm.empty:
         # Ensure '总边际贡献' exists, calculate if necessary (based on production cost only CM)
        if '单位边际贡献 (仅生产成本)' not in factory_production_for_cm.columns:
             # Merge with unit_contribution_margin_prod_cost_df to get the necessary column
            factory_production_for_cm = pd.merge(
                 factory_production_for_cm,
                 unit_contribution_margin_prod_cost_df[['工厂', '生产线', '产品', '单位边际贡献 (仅生产成本)']],
                 on=['工厂', '生产线', '产品'],
                 how='left'
             )
             # Fill NaN CM values with 0 (e.g., if a product was produced but no CM data)
            factory_production_for_cm['单位边际贡献 (仅生产成本)'].fillna(0, inplace=True)
             # Recalculate total contribution margin based on production cost only
            factory_production_for_cm['总边际贡献'] = factory_production_for_cm['生产量'] * factory_production_for_cm['单位边际贡献 (仅生产成本)']


        total_cm_factory = factory_production_for_cm['总边际贡献'].sum()
        total_prod_factory = factory_production_for_cm['生产量'].sum()
        weighted_avg_cm_factory = total_cm_factory / (total_prod_factory + epsilon) if total_prod_factory > 0 else 0
    else:
        weighted_avg_cm_factory = 0


    # Calculate break-even quantity for the factory (approximation)
    break_even_quantity_factory = total_activation_cost_factory / (weighted_avg_cm_factory + epsilon) if weighted_avg_cm_factory > 0 else float('inf')

    break_even_factory_prod_act_cost_data.append({
        '工厂': f,
        '总生产线激活成本': total_activation_cost_factory,
        '加权平均单位边际贡献': weighted_avg_cm_factory,
        '工厂盈亏平衡产量 (件, 仅生产+激活成本)': break_even_quantity_factory
    })

break_even_factory_prod_act_cost_df = pd.DataFrame(break_even_factory_prod_act_cost_data)

print("\n--- 工厂盈亏平衡产量 (仅考虑生产+激活成本, 按工厂汇总) ---")
if not break_even_factory_prod_act_cost_df.empty:
    display(break_even_factory_prod_act_cost_df)
else:
    print("未能计算工厂盈亏平衡产量（仅考虑生产+激活成本）。")


# Summarize Overall (sum of all activation costs, weighted average CM)
total_overall_activation_cost = sum(line_activation_cost.values())

# Calculate weighted average unit CM for overall based on total production volume
# Need total production volume and total contribution margin (production cost only) overall
if not production_plan_with_cm.empty:
     # Ensure '总边际贡献' exists, calculate if necessary (based on production cost only CM)
    if '单位边际贡献 (仅生产成本)' not in production_plan_with_cm.columns:
         # Merge with unit_contribution_margin_prod_cost_df to get the necessary column
        production_plan_with_cm = pd.merge(
             production_plan_with_cm,
             unit_contribution_margin_prod_cost_df[['工厂', '生产线', '产品', '单位边际贡献 (仅生产成本)']],
             on=['工厂', '生产线', '产品'],
             how='left'
         )
         # Fill NaN CM values with 0
        production_plan_with_cm['单位边际贡献 (仅生产成本)'].fillna(0, inplace=True)
         # Recalculate total contribution margin based on production cost only
        production_plan_with_cm['总边际贡献'] = production_plan_with_cm['生产量'] * production_plan_with_cm['单位边际贡献 (仅生产成本)']

    total_cm_overall = production_plan_with_cm['总边际贡献'].sum()
    total_prod_overall = production_plan_with_cm['生产量'].sum()
    weighted_avg_cm_overall = total_cm_overall / (total_prod_overall + epsilon) if total_prod_overall > 0 else 0
else:
    weighted_avg_cm_overall = 0


# Calculate break-even quantity overall (approximation)
break_even_quantity_overall = total_overall_activation_cost / (weighted_avg_cm_overall + epsilon) if weighted_avg_cm_overall > 0 else float('inf')

break_even_overall_prod_act_cost_data = [{
    '总生产线激活成本': total_overall_activation_cost,
    '加权平均单位边际贡献': weighted_avg_cm_overall,
    '全部汇总盈亏平衡产量 (件, 仅生产+激活成本)': break_even_quantity_overall
}]

break_even_overall_prod_act_cost_df = pd.DataFrame(break_even_overall_prod_act_cost_data)

print("\n--- 全部汇总盈亏平衡产量 (仅考虑生产+激活成本) ---")
if not break_even_overall_prod_act_cost_df.empty:
    display(break_even_overall_prod_act_cost_df)
else:
    print("未能计算全部汇总盈亏平衡产量（仅考虑生产+激活成本）。")


--- 生产线盈亏平衡产量 (仅考虑生产+激活成本, 按工厂、生产线、周期) ---


,工厂,生产线,周期（旬）,生产线激活成本,平均单位边际贡献 (仅生产成本),"盈亏平衡产量 (件, 仅生产+激活成本)"
0,北京工厂,Line_1,1,5000,3.7,1351.351351
1,北京工厂,Line_1,2,5000,3.7,1351.351351
2,北京工厂,Line_2,1,4800,3.7,1297.297297
3,北京工厂,Line_2,2,4800,3.7,1297.297297
4,广州工厂,Line_A,1,4500,4.0,1125.000000
5,广州工厂,Line_A,2,4500,4.0,1125.000000
6,广州工厂,Line_A,3,4500,4.0,1125.000000
7,广州工厂,Line_B,1,4000,3.6,1111.111111
8,广州工厂,Line_B,2,4000,3.6,1111.111111
9,上海工厂,Line_X,1,6000,3.6,1666.666666


NameError: name 'production_plan_with_cm' is not defined

In [27]:
# Re-run the Visualize Break-even Quantity (Production and Activation Cost Only) cell
# --- 可视化盈亏平衡产量 (仅考虑生产成本和激活成本) ---

if not break_even_prod_act_cost_df.empty:
    fig_be_prod_act_line = px.bar(
        break_even_prod_act_cost_df,
        x='周期（旬）',
        y='盈亏平衡产量 (件, 仅生产+激活成本)',
        color='生产线',
        facet_col='工厂',
        title='生产线盈亏平衡产量（仅考虑生产+激活成本，按工厂和周期）',
        labels={'盈亏平衡产量 (件, 仅生产+激活成本)': '盈亏平衡产量 (件)', '周期（旬）': '周期（旬)'}
    )
    fig_be_prod_act_line.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig_be_prod_act_line.show()
else:
    print("没有盈亏平衡产量（仅考虑生产+激活成本）数据可供可视化。")

# 可视化按工厂汇总的盈亏平衡产量 (仅考虑生产+激活成本)
if not break_even_factory_prod_act_cost_df.empty:
    fig_be_prod_act_factory = px.bar(
        break_even_factory_prod_act_cost_df,
        x='工厂',
        y='工厂盈亏平衡产量 (件, 仅生产+激活成本)',
        title='工厂盈亏平衡产量（仅考虑生产+激活成本）',
        labels={'工厂盈亏平衡产量 (件, 仅生产+激活成本)': '盈亏平衡产量 (件)', '工厂': '工厂'}
    )
    fig_be_prod_act_factory.show()
else:
    print("没有工厂盈亏平衡产量（仅考虑生产+激活成本）数据可供可视化。")

# 可视化全部汇总的盈亏平衡产量 (仅考虑生产+激活成本)
if not break_even_overall_prod_act_cost_df.empty:
    fig_be_prod_act_overall = px.bar(
        break_even_overall_prod_act_cost_df,
        x=break_even_overall_prod_act_cost_df.index, # Use index as x-axis for single row
        y='全部汇总盈亏平衡产量 (件, 仅生产+激活成本)',
        title='全部汇总盈亏平衡产量（仅考虑生产+激活成本）',
        labels={'全部汇总盈亏平衡产量 (件, 仅生产+激活成本)': '盈亏平衡产量 (件)'}
    )
    fig_be_prod_act_overall.update_xaxes(visible=False) # Hide x-axis as it's just an index
    fig_be_prod_act_overall.show()
else:
    print("没有全部汇总盈亏平衡产量（仅考虑生产+激活成本）数据可供可视化。")

NameError: name 'break_even_factory_prod_act_cost_df' is not defined